# langchain learning

## 初始化

In [3]:
# 加载 .env 到环境变量
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

## LCEL

### LCEL说明（翻译原文）

LangChain 表达式语言（LCEL）是一种轻松地将链组合在一起的声明性方式。 LCEL 从第一天起就被设计为支持将原型投入生产，无需更改代码，从最简单的“提示 + LLM”链到最复杂的链（我们已经看到人们在生产中成功运行了 100 个步骤的 LCEL 链）。 强调一下您可能想要使用 LCEL 的一些原因：

流支持当您使用 LCEL 构建链时，您可以获得最佳的首次代币时间（直到第一个输出块出现之前经过的时间）。 对于某些连锁店来说，这意味着例如。 我们将令牌直接从 LLM 流式传输到流式输出解析器，然后您会以与 LLM 提供者输出原始令牌相同的速率返回已解析的增量输出块。

异步支持使用 LCEL 构建的任何链都可以使用同步 API（例如，在原型设计时在 Jupyter 笔记本中）和异步 API（例如，在 LangServe 服务器中）进行调用。 这使得能够在原型和生产中使用相同的代码，具有出色的性能，并且能够在同一服务器中处理许多并发请求。

优化的并行执行 只要您的 LCEL 链具有可以并行执行的步骤（例如，如果您从多个检索器获取文档），我们就会在同步和异步接口中自动执行此操作，以尽可能减少延迟。

重试和回退 为 LCEL 链的任何部分配置重试和回退。 这是让您的链条在规模上更加可靠的好方法。 我们目前正在努力添加对重试/回退的流支持，以便您可以获得更高的可靠性，而无需任何延迟成本。

访问中间结果对于更复杂的链，即使在生成最终输出之前访问中间步骤的结果通常也非常有用。 这可以用来让最终用户知道正在发生的事情，甚至只是为了调试您的链。 您可以流式传输中间结果，并且它在每个 LangServe 服务器上都可用。

输入和输出模式 输入和输出模式为每个 LCEL 链提供从链结构推断出的 Pydantic 和 JSONSchema 模式。 这可用于验证输入和输出，并且是 LangServe 的组成部分。

### 基本例子

#### prompt + model + output parse

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI(model="gpt-3.5-turbo-1106")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})

'Why did the ice cream go to therapy? Because it was feeling a little sundae-pressed!'

#### prompt

Prompt 是一个 BasePromptTemplate，这意味着它接受模板变量的字典并生成 PromptValue。 
PromptValue 是完整提示的包装器，可以传递给 LLM（将字符串作为输入）或 ChatModel（将一系列消息作为输入）。 
它可以使用任一语言模型类型，因为它定义了用于生成 BaseMessage 和生成字符串的逻辑。

In [5]:
prompt_value = prompt.invoke({"topic": "ice cream"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

#### model

然后将 PromptValue 传递给模型。 在本例中，我们的模型是 ChatModel，这意味着它将输出 BaseMessage。

In [6]:
message = model.invoke(prompt_value)
message

AIMessage(content='Why did the ice cream go to therapy? Because it had too many sprinkles of anxiety!')

In [7]:
# 如果不是对话模型，而是LLM模型，就会输出字符串

from langchain_openai.llms import OpenAI
llm = OpenAI(model="gpt-3.5-turbo-instruct")
llm.invoke(prompt_value)

' \n\nAI: Why did the ice cream go to therapy? Because it had a rocky road!'

#### Output parser

最后，我们将模型输出传递给 output_parser，它是一个 BaseOutputParser，这意味着它接受字符串或 BaseMessage 作为输入。
StrOutputParser 特别简单地将任何输入转换为字符串。

In [8]:
output_parser.invoke(message)

'Why did the ice cream go to therapy? Because it had too many sprinkles of anxiety!'

#### 整个Pipeline

请按照以下步骤操作：

- 我们将所需主题的用户输入传递为 {"topic": "ice Cream"}
- 提示组件接受用户输入，然后在使用主题构造提示后使用该输入构造 PromptValue。
- 模型组件采用生成的提示，并传递到 OpenAI LLM 模型进行评估。 模型生成的输出是 ChatMessage 对象。
- 最后，output_parser 组件接收 ChatMessage，并将其转换为 Python 字符串，该字符串从 invoke 方法返回。

In [9]:
input = {"topic": "ice cream"}

prompt.invoke(input)
# > ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

(prompt | model).invoke(input)
# > AIMessage(content="Why did the ice cream go to therapy?\nBecause it had too many toppings and couldn't cone-trol itself!")

AIMessage(content='Why did the ice cream go to therapy? Because it had too many sprinkles of anxiety!')

### RAG的例子

In [21]:
!poetry add "langchain[docarray]"

Using version ^0.1.3 for langchain

Updating dependencies
Resolving dependencies... (14.7s)Resolving dependencies... (9.5s)Resolving dependencies... (9.6s)

No dependencies to install or update


In [ ]:
# Requires:
# pip install langchain docarray tiktoken

from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("where did harrison work?")

### 为何使用LCEL

LCEL 可以轻松地从基本组件构建复杂的链条。 

它通过提供以下功能来实现此目的： 
1. 统一的接口：每个 LCEL 对象都实现 Runnable 接口，该接口定义一组通用的调用方法（invoke、batch、stream、ainvoke 等）。 这使得 LCEL 对象链也可以自动支持这些调用。 也就是说，每个 LCEL 对象链本身就是一个 LCEL 对象。
2. 组合原语：LCEL 提供了许多原语，可以轻松组合链、并行化组件、添加后备、动态配置链内部等。


### chain的接口

#### Runnable 协议

要支持链式调用，就必须实现 **Runnable** 协议。 
这是一个标准接口，可以轻松定义自定义链并以标准方式调用它们。 

标准接口包括：

- stream：流回响应块
- invoke：在输入上调用链
- batch：在输入列表上调用链

这些也有相应的异步方法：

- astream：异步响应的流回块
- ainvoke：在输入异步上调用链
- abatch：异步调用输入列表上的链
- astream_log：除了最终响应之外，还流回发生的中间步骤
- astream_events：链中发生的流事件（在 langchain-core 0.1.14 中引入）

不同组件的输入输出：

| 组件名称    | 输入类型   | 输出类型     |
|-----------|------------|-------------|
| Prompt    | 字典 | PromptValue |
| ChatModel | 字符串，对话消息或PrompValue | ChatMessage |
| LLM       | 字符串，对话消息或PrompValue | 字符串 |
| OutputParser | LLM 或 ChatModel 的输出 | 依赖于解析器的实现 |
| Retriever | 字符串 | 文档列表 |
| Tool      | 字符串或字典（依赖于工具的实现） | 依赖于工具的实现 |

#### 输入输出schema

**chain**的输入输出都使用**pydantic**来检验**schema**是否合规。

In [26]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
chain = prompt | model

In [32]:
prompt.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [36]:
prompt.output_schema.schema()

{'title': 'ChatPromptTemplateOutput',
 'anyOf': [{'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'AIMessage': {'title': 'AIMessage',
   'description': 'A Message from an AI.',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': 'string'},
      {'type': 'array',
       'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]}}]},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'type': {'title': 'Type',
     'default': 'ai',
     'enum': ['ai'],
     'type': 'string'},
    'example': {'title': 'Example', 'default': F

In [27]:
# The input schema of the chain is the input schema of its first part, the prompt.
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [29]:
model.input_schema.schema()

{'title': 'ChatOpenAIInput',
 'anyOf': [{'type': 'string'},
  {'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'},
  {'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'},
     {'$ref': '#/definitions/HumanMessage'},
     {'$ref': '#/definitions/ChatMessage'},
     {'$ref': '#/definitions/SystemMessage'},
     {'$ref': '#/definitions/FunctionMessage'},
     {'$ref': '#/definitions/ToolMessage'}]}}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'AIMessage': {'title': 'AIMessage',
   'description': 'A Message from an AI.',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': 'str

In [35]:
model.output_schema.schema()

{'title': 'ChatOpenAIOutput',
 'anyOf': [{'$ref': '#/definitions/AIMessage'},
  {'$ref': '#/definitions/HumanMessage'},
  {'$ref': '#/definitions/ChatMessage'},
  {'$ref': '#/definitions/SystemMessage'},
  {'$ref': '#/definitions/FunctionMessage'},
  {'$ref': '#/definitions/ToolMessage'}],
 'definitions': {'AIMessage': {'title': 'AIMessage',
   'description': 'A Message from an AI.',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': 'string'},
      {'type': 'array',
       'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]}}]},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'type': {'title': 'Type',
     'default': 'ai',
     'enum': ['ai'],
     'type': 'string'},
    'example': {'title': 'Example', 'default': False, 'type': 'boolean'}},
   'required': ['content']},
  'HumanMessage': {'title': 'HumanMessage',
   'description': 'A Message from a human.',
   'type': 'object',
   'properties': {'conten

In [30]:
# The output schema of the chain is the output schema of its last part, in this case a ChatModel, which outputs a ChatMessage
chain.output_schema.schema()

{'title': 'ChatOpenAIOutput',
 'anyOf': [{'$ref': '#/definitions/AIMessage'},
  {'$ref': '#/definitions/HumanMessage'},
  {'$ref': '#/definitions/ChatMessage'},
  {'$ref': '#/definitions/SystemMessage'},
  {'$ref': '#/definitions/FunctionMessage'},
  {'$ref': '#/definitions/ToolMessage'}],
 'definitions': {'AIMessage': {'title': 'AIMessage',
   'description': 'A Message from an AI.',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': 'string'},
      {'type': 'array',
       'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]}}]},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'type': {'title': 'Type',
     'default': 'ai',
     'enum': ['ai'],
     'type': 'string'},
    'example': {'title': 'Example', 'default': False, 'type': 'boolean'}},
   'required': ['content']},
  'HumanMessage': {'title': 'HumanMessage',
   'description': 'A Message from a human.',
   'type': 'object',
   'properties': {'conten

#### 接口调用

In [37]:
for s in chain.stream({"topic": "bears"}):
    print(s.content, end="", flush=True)

Sure! Here's a bear joke for you:

Why don't bears use cellphones?

Because they already have paws for "bear-y" good reception!

In [ ]:
chain.invoke({"topic": "bears"})

#### 并发调用

LCEL支持并发调用，例如使用 **RunnableParallel**：

In [43]:
from langchain_core.runnables import RunnableParallel

chain1 = ChatPromptTemplate.from_template("告诉我一个关于{topic}的笑话") | model
chain2 = (
    ChatPromptTemplate.from_template("写一首关于{topic}的两行短诗")
    | model
)
combined = RunnableParallel(joke=chain1, poem=chain2)

In [44]:
%%time
chain1.batch([{"topic": "熊"}, {"topic": "猫"}])

CPU times: user 36.9 ms, sys: 5.02 ms, total: 42 ms
Wall time: 7.74 s


[AIMessage(content='为什么熊不喜欢使用电脑？\n因为它们总是会误键Paw（爪子）！'),
 AIMessage(content='当然！这是一个关于猫的笑话：\n\n有一天，一个猫咪走进了一个酒吧，走到吧台前坐下，对着酒保说：“老板，给我来一杯牛奶，不加冰块。” \n\n酒保看着猫咪惊讶地问道：“哇，我还没见过一只会喝牛奶的猫咪呢！你是怎么学会喝牛奶的？” \n\n猫咪慢慢地回答：“其实，我一开始也不会喝牛奶。有一天，我看见一只狗喝牛奶，他告诉我喝牛奶可以让我变得更强壮。” \n\n酒保好奇地问：“那结果呢？你变得更强壮了吗？” \n\n猫咪嘟起嘴巴，苦笑着说：“没有啊！我只变得更胖了！”')]

In [45]:
%%time
chain2.batch([{"topic": "熊"}, {"topic": "猫"}])

CPU times: user 33.5 ms, sys: 4.98 ms, total: 38.5 ms
Wall time: 2.17 s


[AIMessage(content='毛茸茸熊宝宝，温暖心房笑声多。'), AIMessage(content='懒卧红蓝瓦，猫眼如夜星辉华。')]

In [46]:
%%time
combined.batch([{"topic": "熊"}, {"topic": "猫"}])

CPU times: user 78.1 ms, sys: 7.97 ms, total: 86 ms
Wall time: 4.15 s


[{'joke': AIMessage(content='当然！这是一个关于熊的笑话：\n\n有一天，一只熊走进了一家餐馆。熊走上吧台，对服务员说：“我想要一杯可乐和......一份烤鸡翅。”\n服务员非常惊讶，但还是问熊：“为什么大熊要来这家餐馆点餐？”\n熊回答说：“因为我听说这是唯一一家不加鱼的餐馆！”'),
  'poem': AIMessage(content='熊儿啊，胸怀无尽温柔。\n森林中，守护着自然的宝库。')},
 {'joke': AIMessage(content='为什么猫喜欢玩电脑？因为它们喜欢捉鼠标！'),
  'poem': AIMessage(content='柔软小猫咪，眼中闪亮星辰')}]

### 实践技巧

#### RunnableParallel（并行处理输入输出）

输入：**RunnableParallel** 对于操作一个 **Runnable** 的输出以匹配序列中下一个 **Runnable** 的输入格式非常有用。

请注意，当将 **RunnableParallel** 与另一个 **Runnable** 组合时，
我们甚至不需要将字典包装在 **RunnableParallel** 类中 - 类型转换已为我们处理。 
在链的上下文中，这些是等效的：

In [ ]:
## 以KV形式直接在链的上下文中做类型转换
{"context": retriever, "question": RunnablePassthrough()}
## KV参数构造
RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
## 逐个参数构造
RunnableParallel(context=retriever, question=RunnablePassthrough())

使用 **itemgetter** 提取尚未分配的值：

In [ ]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question": "where did harrison work", "language": "italian"})

输出：也可以实现多个 **chain** 的并发调用，并返回一个键值。

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
joke_chain = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
poem_chain = (
    ChatPromptTemplate.from_template("write a 2-line poem about {topic}") | model
)

map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)

map_chain.invoke({"topic": "bear"})

#### RunnablePassthrough（额外输入）

In [48]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

#### RunnableLambda（惰性函数）

In [ ]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI


def length_function(text):
    return len(text)


def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)


def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])


prompt = ChatPromptTemplate.from_template("what is {a} + {b}")
model = ChatOpenAI()

chain1 = prompt | model

chain = (
    {
        "a": itemgetter("foo") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
)

In [ ]:
chain.invoke({"foo": "bar", "bar": "gah"})

#### RunnableBranch（动态路由）

**RunnableBranch** 使用（条件、可运行）对列表和默认可运行来初始化。
它通过传递调用它的输入的每个条件来选择哪个分支。 
它选择第一个条件来评估为 True，并使用输入运行与该条件相对应的可运行程序。

如果没有提供的条件匹配，它将运行默认的可运行程序。

In [ ]:
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    (lambda x: "anthropic" in x["topic"].lower(), anthropic_chain),
    (lambda x: "langchain" in x["topic"].lower(), langchain_chain),
    general_chain,
)

In [ ]:
full_chain = {"topic": chain, "question": lambda x: x["question"]} | branch

In [ ]:
full_chain.invoke({"question": "how do I use Anthropic?"})

同样的功能也可以使用自定义函数：

In [ ]:
def route(info):
    if "anthropic" in info["topic"].lower():
        return anthropic_chain
    elif "langchain" in info["topic"].lower():
        return langchain_chain
    else:
        return general_chain

In [ ]:
from langchain_core.runnables import RunnableLambda

full_chain = {"topic": chain, "question": lambda x: x["question"]} | RunnableLambda(route)

#### 模型bind函数（动态传递参数：既不在chain的上游组件中，也不是用户输入的）

In [ ]:
# 传递stop参数
runnable = (
    {"equation_statement": RunnablePassthrough()}
    | prompt
    | model.bind(stop="SOLUTION")
    | StrOutputParser()
)
print(runnable.invoke("x raised to the third plus seven equals 12"))

In [ ]:
# 绑定gpt函数（Need gpt-4 to solve this one correctly）
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Write out the following equation using algebraic symbols then solve it.",
        ),
        ("human", "{equation_statement}"),
    ]
)
model = ChatOpenAI(model="gpt-4", temperature=0).bind(
    function_call={"name": "solver"}, functions=[function]
)
runnable = {"equation_statement": RunnablePassthrough()} | prompt | model
runnable.invoke("x raised to the third plus seven equals 12")

#### 链的配置

#### 使用@chain装饰器创建runnable

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain
from langchain_openai import ChatOpenAI

In [ ]:
prompt1 = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
prompt2 = ChatPromptTemplate.from_template("What is the subject of this joke: {joke}")

In [ ]:
@chain
def custom_chain(text):
    prompt_val1 = prompt1.invoke({"topic": text})
    output1 = ChatOpenAI().invoke(prompt_val1)
    parsed_output1 = StrOutputParser().invoke(output1)
    chain2 = prompt2 | ChatOpenAI() | StrOutputParser()
    return chain2.invoke({"joke": parsed_output1})

In [ ]:
custom_chain.invoke("bears")

#### fallback

#### 流式输出

#### 检查runnable的方法

#### 增加消息历史

## 模型/IO封装

### 快速开始

In [19]:
# 请使用最新的open包
!pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.5/229.5 kB 455.3 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 2.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 28.9 kB/s eta 0:00:00a 0:00:01m
  Attempting uninstall: tiktoken
    Found existing installation: tiktoken 0.4.0
    Uninstalling tiktoken-0.4.0:
      Successfully uninstalled tiktoken-0.4.0
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.0.75
    Uninstalling langsmith-0.0.75:
      Successfully uninstalled langsmith-0.0.75
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.4
    Uninstalling langchain-core-0.1.4:
      Successfully uninstalled langchain-core-0.1.4


#### LLM

In [20]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI

llm = OpenAI()
chat_model = ChatOpenAI()

In [35]:
text = "请帮我想一想，生产彩色铅笔的公司有什么好名字?"
llm.invoke(text)

'\n\n1. 彩虹铅笔厂 \n2. 色彩铅笔工厂 \n3. 绚丽铅笔生产厂 \n4. 七彩铅笔公司 \n5. 彩绘铅笔制造厂 \n6. 色彩笔芯工坊 \n7. 色彩之星铅笔厂 \n8. 彩色艺术笔厂 \n9. 魔法彩铅笔厂 \n10. 色彩创意铅笔厂'

In [36]:
from langchain.schema import HumanMessage
messages = [HumanMessage(content=text)]
chat_model.invoke(messages)

AIMessage(content='1. 艳彩铅笔\n2. 彩绘铅笔\n3. 彩虹铅笔\n4. 色彩世界\n5. 彩铅工坊\n6. 炫彩铅笔\n7. 彩色艺术\n8. 色彩创意\n9. 彩虹创造\n10. 彩笔之家')

#### Prompt

In [34]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("请帮我想一想，生产{product}的公司有什么好名字?")
prompt.format(product="彩色铅笔")

'请帮我想一想，生产彩色铅笔的公司有什么好名字?'

In [26]:
from langchain.prompts.chat import ChatPromptTemplate

template = "You are a helpful assistant that translates {input_language} to {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

chat_prompt.format_messages(input_language="English", output_language="French", text="I love programming.")

[SystemMessage(content='You are a helpful assistant that translates English to French.'),
 HumanMessage(content='I love programming.')]

#### Output parsers

In [43]:
from langchain.schema.output_parser import StrOutputParser

output_parser = CommaSeparatedListOutputParser()
output_parser.parse("hi, bye")

['hi', 'bye']

#### 使用LCEL

In [32]:
template = "生成5个关于{text}的列表 {text}.\n\n{format_instructions}"

chat_prompt = ChatPromptTemplate.from_template(template)
chat_prompt = chat_prompt.partial(format_instructions=output_parser.get_format_instructions())
chain = chat_prompt | chat_model | output_parser
chain.invoke({"text": "colors"})

['red', 'blue', 'yellow', 'green', 'orange']

### 主要概念

#### 模型

请记住：OpenAI模型对提示语中包含JSON的情况非常友好。

#### 消息

- HumanMessage： 一般是纯文字内容
- AIMessage： 可能包含additional_kwargs，例如 funciton calling 提示
- SystemMessage：部份模型支持的内容提示
- FunctionMessage：函数调用的名称和参数
- ToolMessage：工具调用结果（与FunctionMessage不同）

#### 提示语

- PromptValue
- PromptTemplate
- MessagePromptTemplate
- MessagesPlaceholder
- ChatPromptTemplate

#### Output Parsers

- StrOutputParser：仅输出字符串；如果输出是 ChatModel，它会仅输出Message的content属性
- OpenAI Functions Parsers：处理OpenAI函数调用所需的函数名和参数
- Agent Output Parsers：帮助智能体解析执行计划

In [50]:
from langchain.schema.output_parser import StrOutputParser
parser = StrOutputParser()
response = chat_prompt.format_messages(input_language="English", output_language="French", text="I love programming.")
print(response)
print(parser.parse(response))

[HumanMessage(content='生成5个关于I love programming.的列表 I love programming..\n\nYour response should be a list of comma separated values, eg: `foo, bar, baz`')]
[HumanMessage(content='生成5个关于I love programming.的列表 I love programming..\n\nYour response should be a list of comma separated values, eg: `foo, bar, baz`')]


### Prompt封装

In [51]:
# 简单的例子
from langchain.prompts import PromptTemplate

template = PromptTemplate.from_template("给我讲个关于{subject}的笑话")
print(template)
print(template.format(subject='小明'))

input_variables=['subject'] template='给我讲个关于{subject}的笑话'
给我讲个关于小明的笑话


#### 组装提示语（字符串模板）

使用字符串提示时，每个模板都会连接在一起。
您可以直接使用prompt模板或字符串（但列表中的第一个元素必须是prompt模板类型）。

In [52]:
from langchain.prompts import PromptTemplate

In [55]:
# 提示语模板与字符串可以直接相加，简化模板构造
prompt = (
    PromptTemplate.from_template("Tell me a joke about {topic}")
    + ", make it funny"
    + "\n\nand in {language}"
)
prompt

PromptTemplate(input_variables=['language', 'topic'], template='Tell me a joke about {topic}, make it funny\n\nand in {language}')

In [58]:
# 这是一个完整的函数调用，达到同样的效果
PromptTemplate(
    input_variables=['language', 'topic'],
    output_parser=None,
    partial_variables={},
    template='Tell me a joke about {topic}, make it funny\n\nand in {language}',
    template_format='f-string',
    validate_template=True
)

PromptTemplate(input_variables=['language', 'topic'], template='Tell me a joke about {topic}, make it funny\n\nand in {language}', validate_template=True)

In [60]:
prompt.format(topic="sports", language="chinese")

'Tell me a joke about sports, make it funny\n\nand in chinese'

In [67]:
# 也可以直接在chain中给参数
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
chain = LLMChain(llm=model, prompt=prompt,output_parser=parser)

chain.invoke({"topic": "sports", "language": "chinse"})

{'topic': 'sports',
 'language': 'chinse',
 'text': '为什么足球场上的草总是那么自信？\n因为它知道自己会被球员踩！'}

#### 组装提示语（对话模式）

In [68]:
from langchain.schema import AIMessage, HumanMessage, SystemMessage
prompt = SystemMessage(content="You are a nice pirate")
new_prompt = (
    prompt + HumanMessage(content="hi") + AIMessage(content="what?") + "{input}"
)
new_prompt.format_messages(input="i said hi")

[SystemMessage(content='You are a nice pirate'),
 HumanMessage(content='hi'),
 AIMessage(content='what?'),
 HumanMessage(content='i said hi')]

#### 在提示语中填充例子

#### 在提示语中填充小样本

#### 局部修改提示语模板

In [80]:
# 通过局部修改实现提示语管理
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(template="{foo}{bar}", input_variables=["foo", "bar"])
partial_prompt = prompt.partial(foo="foo")
print(partial_prompt.format(bar="baz"))

foobaz


In [81]:
# 或者这样做
prompt = PromptTemplate(
    template="{foo}{bar}", input_variables=["bar"], partial_variables={"foo": "foo"}
)
print(prompt.format(bar="baz"))

foobaz


In [82]:
# 使用函数
from datetime import datetime

def _get_datetime():
    now = datetime.now()
    return now.strftime("%m/%d/%Y, %H:%M:%S")

prompt = PromptTemplate(
    template="Tell me a {adjective} joke about the day {date}",
    input_variables=["adjective", "date"],
)
partial_prompt = prompt.partial(date=_get_datetime)
print(partial_prompt.format(adjective="funny"))


Tell me a funny joke about the day 01/23/2024, 16:27:57


In [83]:
# 换个方式使用函数
prompt = PromptTemplate(
    template="Tell me a {adjective} joke about the day {date}",
    input_variables=["adjective"],
    partial_variables={"date": _get_datetime},
)
print(prompt.format(adjective="funny"))

Tell me a funny joke about the day 01/23/2024, 16:28:51


#### 提示语pipeline

In [69]:
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts.prompt import PromptTemplate

In [70]:
full_template = """{introduction}

{example}

{start}"""
full_prompt = PromptTemplate.from_template(full_template)

In [71]:
introduction_template = """You are impersonating {person}."""
introduction_prompt = PromptTemplate.from_template(introduction_template)

In [72]:
example_template = """Here's an example of an interaction:

Q: {example_q}
A: {example_a}"""
example_prompt = PromptTemplate.from_template(example_template)

In [73]:
start_template = """Now, do this for real!

Q: {input}
A:"""
start_prompt = PromptTemplate.from_template(start_template)

In [75]:
input_prompts = [
    ("introduction", introduction_prompt),
    ("example", example_prompt),
    ("start", start_prompt),
]
pipeline_prompt = PipelinePromptTemplate(
    final_prompt=full_prompt, pipeline_prompts=input_prompts
)

In [76]:
pipeline_prompt.input_variables

['person', 'example_a', 'example_q', 'input']

In [77]:
print(
    pipeline_prompt.format(
        person="Elon Musk",
        example_q="What's your favorite car?",
        example_a="Tesla",
        input="What's your favorite social media site?",
    )
)

You are impersonating Elon Musk.

Here's an example of an interaction:

Q: What's your favorite car?
A: Tesla

Now, do this for real!

Q: What's your favorite social media site?
A:


### 对话模型

#### LCEL

对话模型实现了Runnable接口，并自动实现以下接口：

- invoke
- ainvoke
- stream
- astream
- batch
- abatch
- astream_log

In [86]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

chat = ChatOpenAI()
messages = [
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(content="模型为什么要做正则化?"),
]
chat.invoke(messages)

AIMessage(content='模型在训练数据上表现良好，但在新的未见过的数据上可能会出现过拟合（overfitting）的情况。正则化是一种用来解决过拟合问题的技术。它通过在损失函数中引入一个正则化项，限制模型的复杂度，防止模型过度拟合训练数据。\n\n正则化的目的是平衡模型的拟合能力和泛化能力。如果模型过于复杂，它可能会过度拟合训练数据，导致在新数据上的表现较差。正则化通过对模型参数的惩罚，鼓励模型选择更简单的参数组合，从而降低模型的复杂度。\n\n常见的正则化方法包括L1正则化和L2正则化。L1正则化通过在损失函数中添加模型参数的绝对值之和，促使模型参数稀疏化，即让一些参数变为0，从而实现特征选择的效果。L2正则化通过在损失函数中添加模型参数的平方和，降低参数的绝对值，使模型更加平滑。\n\n正则化可以帮助减少模型的方差，提高模型的泛化能力，从而在新的未见过的数据上表现更好。它是训练模型时常用的一种技术，可以提高模型的稳定性和可靠性。')

In [88]:
for chunk in chat.stream(messages):
    print(chunk.content, end="", flush=True)

模型正则化是为了减少过拟合（Overfitting）的发生。在训练模型时，如果模型过于复杂，容易出现过拟合的情况，即在训练集上表现很好，但在未知数据上表现较差。过拟合的原因是因为模型过度拟合了训练数据的噪声和细节，并且没有很好地学习到数据的普遍规律。

正则化通过在模型的损失函数中引入正则项，对模型的复杂度进行惩罚，从而降低模型的复杂度。正则化的目的是通过控制模型参数的大小，使模型更加简单，能够更好地泛化到未知数据上。常见的正则化方法有L1正则化和L2正则化。

L1正则化通过在损失函数中添加模型参数的L1范数（绝对值之和）作为正则项，可以使得模型的部分参数变为0，从而实现特征选择的效果，减少模型的复杂度。

L2正则化通过在损失函数中添加模型参数的L2范数（平方和的平方根）作为正则项，可以使得模型参数的值较小，从而降低模型的复杂度。

正则化可以在一定程度上防止过拟合，提高模型的泛化能力，使模型在未知数据上表现更好。

In [89]:
chat.batch([messages])

[AIMessage(content='模型正则化是为了解决过拟合问题。过拟合是指模型在训练数据上表现良好，但在新的未见过的数据上表现较差的现象。正则化通过在模型的损失函数中添加一个正则化项，惩罚模型的复杂度，从而限制模型的学习能力，减少模型对训练数据的过度拟合。\n\n正则化有助于提高模型的泛化能力，使其在新数据上的表现更好。常见的正则化方法包括L1正则化（Lasso）和L2正则化（Ridge），它们分别通过对模型的权重进行惩罚，降低模型的复杂度。正则化还可以用于特征选择，通过对特征的权重进行惩罚，减少对不相关特征的依赖。\n\n总之，模型正则化是为了防止过拟合，提高模型的泛化能力，从而使模型在未见过的数据上表现更好。')]

#### 使用内存缓存

In [90]:
from langchain.globals import set_llm_cache
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [100]:
%%time
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())

# The first time, it is not yet in cache, so it should take longer
llm.invoke("Tell me a joke")

CPU times: user 11.2 ms, sys: 2.09 ms, total: 13.3 ms
Wall time: 1.98 s


AIMessage(content="Sure, here's a classic one for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!")

下面的相同调用不会重复访问大模型：

In [102]:
%%time
# The second time it is, so it goes faster
llm.invoke("Tell me a joke")

CPU times: user 1.24 ms, sys: 101 µs, total: 1.34 ms
Wall time: 2.09 ms


AIMessage(content="Sure, here's a classic one for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!")

#### 使用SQLite缓存

In [103]:
!rm .langchain.db

rm: .langchain.db: No such file or directory


In [104]:
# We can do the same thing with a SQLite cache
from langchain.cache import SQLiteCache

set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [107]:
%%time
# The first time, it is not yet in cache, so it should take longer
llm.predict("Tell me a joke")

CPU times: user 3.23 ms, sys: 1.27 ms, total: 4.49 ms
Wall time: 3.59 ms


"Sure, here's a classic one for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!"

#### Token跟踪

In [110]:
from langchain.callbacks import get_openai_callback
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
with get_openai_callback() as cb:
    result = llm.invoke("Tell me a new joke")
    print(cb)

llm4 = ChatOpenAI(model_name="gpt-4")
with get_openai_callback() as cb:
    result = llm4.invoke("Tell me a new apple joke")
    print(cb)

Tokens Used: 35
	Prompt Tokens: 12
	Completion Tokens: 23
Successful Requests: 1
Total Cost (USD): $6.4e-05
Tokens Used: 32
	Prompt Tokens: 13
	Completion Tokens: 19
Successful Requests: 1
Total Cost (USD): $0.00153


### LLMs

#### OpenAI封装

In [7]:
# 最简单的代码
from langchain_openai import ChatOpenAI

llm = ChatOpenAI() # 默认是gpt-3.5-turbo
response = llm.invoke("你是谁")
print(response.content)

我是一个AI助手，被称为OpenAI Assistant。我被设计用来回答各种问题和提供帮助。有什么我可以帮助你的吗？


#### 通义千问封装

In [10]:
!poetry add dashscope

Using version ^1.14.0 for dashscope

Updating dependencies
Resolving dependencies... (0.8s)

Package operations: 1 install, 0 updates, 0 removals

  • Installing dashscope (1.14.0): Pending...
  • Installing dashscope (1.14.0): Downloading... 0%
  • Installing dashscope (1.14.0): Downloading... 80%
  • Installing dashscope (1.14.0): Downloading... 100%
  • Installing dashscope (1.14.0): Installing...
  • Installing dashscope (1.14.0)

Writing lock file


In [12]:
# 其它模型分装在 langchain_community 底包中
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import Tongyi

ty_llm = Tongyi()
messages = [
    HumanMessage(content="你是谁") 
]
ty_llm.invoke(messages)

'我是阿里云开发的一款超大规模语言模型，我叫通义千问。'

#### 自定义LLM

In [111]:
from typing import Any, List, Mapping, Optional
from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM

In [118]:
# 实现一个定制的LLM接入
class CustomLLM(LLM):
    n: int

    @property
    def _llm_type(self) -> str:
        return "custom"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")
        return prompt[: self.n]

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {"n": self.n}

In [113]:
llm = CustomLLM(n=10)

In [114]:
llm.invoke("This is a foobar thing")

'This is a '

In [117]:
print(llm)

CustomLLM
Params: {'n': 10}


### 从文件加载提示语模板

#### yaml格式

In [ ]:
 _type: prompt
input_variables:
    ["adjective", "content"]
template: 
    Tell me a {adjective} joke about {content}.

#### json格式

In [16]:
{
    "_type": "prompt",
    "input_variables": ["adjective", "content"],
    "template": "Tell me a {adjective} joke about {content}."
}

{'_type': 'prompt',
 'input_variables': ['adjective', 'content'],
 'template': 'Tell me a {adjective} joke about {content}.'}

#### json + txt

首先，将模板主要内容写入**final_step.txt**：

然后，在**task.json**文件中指定**template_path**嵌入路径：

In [43]:
{
    "_type": "prompt",
    "input_variables": [
      "ai_name",
      "ai_role",
      "task_description",
      "short_term_memory"
    ],
    "template_path": "final_step.txt"
}

{'_type': 'prompt',
 'input_variables': ['ai_name',
  'ai_role',
  'task_description',
  'short_term_memory'],
 'template_path': 'final_step.txt'}

#### 加载提示语模板文件

In [17]:
from langchain.prompts import load_prompt

prompt = load_prompt("simple_prompt.json")
print(prompt.format(adjective="funny", content="Xiao Ming"))

Tell me a funny joke about Xiao Ming.


### OutputParser

自动把 LLM 输出的字符串按指定格式加载。

LangChain 内置的 OutputParser 包括:

- StrOutputParser
- OpenAIFunctions
- ListParser
- DatetimeParser
- EnumParser
- PydanticParser
- XMLParser
等等

#### JSON parser

In [135]:
from typing import List
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0)

# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

In [138]:
# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

print(prompt)
chain.invoke({"query": joke_query})

input_variables=['query'] partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"setup": {"title": "Setup", "description": "question to set up a joke", "type": "string"}, "punchline": {"title": "Punchline", "description": "answer to resolve the joke", "type": "string"}}, "required": ["setup", "punchline"]}\n```'} template='Answer the user query.\n{format_instructions}\n{query}\n'


{'setup': "Why don't scientists trust atoms?",
 'punchline': 'Because they make up everything!'}

#### OpenAI Functions

从 pydantic 转换 openai 函数名和参数：

In [140]:
from langchain_community.utils.openai_functions import (
    convert_pydantic_to_openai_function,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI

class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

openai_functions = [convert_pydantic_to_openai_function(Joke)]

In [141]:
model = ChatOpenAI(temperature=0)
prompt = ChatPromptTemplate.from_messages(
    [("system", "You are helpful assistant"), ("user", "{input}")]
)

**JsonOutputFunctionsParser**

In [144]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
parser = JsonOutputFunctionsParser()

# 绑定openai函数，并使用json解析出参数
chain = prompt | model.bind(functions=openai_functions) | parser

In [143]:
chain.invoke({"input": "tell me a joke"})

{'setup': "Why don't scientists trust atoms?",
 'punchline': 'Because they make up everything!'}

**JsonKeyOutputFunctionsParser**

In [147]:
from typing import List
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

class Jokes(BaseModel):
    """Jokes to tell user."""

    joke: List[Joke]
    funniness_level: int

In [148]:
parser = JsonKeyOutputFunctionsParser(key_name="joke")

In [149]:
openai_functions = [convert_pydantic_to_openai_function(Jokes)]
chain = prompt | model.bind(functions=openai_functions) | parser

In [150]:
chain.invoke({"input": "tell me two jokes"})

[{'setup': "Why don't scientists trust atoms?",
  'punchline': 'Because they make up everything!'},
 {'setup': 'Why did the scarecrow win an award?',
  'punchline': 'Because he was outstanding in his field!'}]

In [151]:
for s in chain.stream({"input": "tell me two jokes"}):
    print(s)

[]
[{}]
[{'setup': ''}]
[{'setup': 'Why'}]
[{'setup': 'Why don'}]
[{'setup': "Why don't"}]
[{'setup': "Why don't scientists"}]
[{'setup': "Why don't scientists trust"}]
[{'setup': "Why don't scientists trust atoms"}]
[{'setup': "Why don't scientists trust atoms?"}]
[{'setup': "Why don't scientists trust atoms?", 'punchline': ''}]
[{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because'}]
[{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they'}]
[{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make'}]
[{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make up'}]
[{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make up everything'}]
[{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make up everything!'}]
[{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make up everything!'}, {}]
[{'setup': "Why don't scientists trust atoms?", 'pu

#### Enum parser

In [130]:
from langchain.output_parsers.enum import EnumOutputParser
from enum import Enum

class Colors(Enum):
    RED = "red"
    GREEN = "green"
    BLUE = "blue"

parser = EnumOutputParser(enum=Colors)

In [131]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

prompt = PromptTemplate.from_template(
    """What color eyes does this person have?

> Person: {person}

Instructions: {instructions}"""
).partial(instructions=parser.get_format_instructions())
chain = prompt | ChatOpenAI() | parser

In [134]:
print(prompt)
chain.invoke({"person": "Frank Sinatra"})

input_variables=['person'] partial_variables={'instructions': 'Select one of the following options: red, green, blue'} template='What color eyes does this person have?\n\n> Person: {person}\n\nInstructions: {instructions}'


<Colors.BLUE: 'blue'>

#### Structured output parser

#### YAML parser

#### XML parser

#### Datetime parser

In [122]:
from langchain.output_parsers import DatetimeOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI

In [123]:
output_parser = DatetimeOutputParser()
template = """Answer the users question:

{question}

{format_instructions}"""
prompt = PromptTemplate.from_template(
    template,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

In [126]:
prompt

PromptTemplate(input_variables=['question'], partial_variables={'format_instructions': "Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 0126-01-12T17:02:00.512595Z, 0719-05-06T14:30:04.335045Z, 1391-07-06T00:25:48.331596Z\n\nReturn ONLY this string, no other words!"}, template='Answer the users question:\n\n{question}\n\n{format_instructions}')

In [129]:
chain = prompt | OpenAI() | output_parser
output = chain.invoke({"question": "when was bitcoin founded?"})
print(output)

2009-01-03 18:15:05


#### Pydantic parser

#### CSV parser

In [119]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

model = ChatOpenAI(temperature=0)

chain = prompt | model | output_parser

In [120]:
chain.invoke({"subject": "ice cream flavors"})

['Vanilla',
 'Chocolate',
 'Strawberry',
 'Mint Chocolate Chip',
 'Cookies and Cream']

In [121]:
for s in chain.stream({"subject": "ice cream flavors"}):
    print(s)

['Vanilla']
['Chocolate']
['Strawberry']
['Mint Chocolate Chip']
['Cookies and Cream']


#### Pandas DataFrame Parser

#### Output-fixing parser

#### Retry parser

## 数据连接封装

最佳实践是为向量数据库提供高质量的问答对。

### 文档加载

#### 文件目录

In [10]:
pip show unstructured

Name: unstructured
Version: 0.12.2
Summary: A library that prepares raw documents for downstream ML tasks.
Home-page: https://github.com/Unstructured-IO/unstructured
Author: Unstructured Technologies
Author-email: devops@unstructuredai.io
License: Apache-2.0
Location: /Users/xuehongwei/Library/Caches/pypoetry/virtualenvs/md-8WLN4Vov-py3.10/lib/python3.10/site-packages
Requires: backoff, beautifulsoup4, chardet, dataclasses-json, emoji, filetype, langdetect, lxml, nltk, numpy, python-iso639, python-magic, rapidfuzz, requests, tabulate, typing-extensions, unstructured-client, wrapt
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!poetry add "unstructured[all-docs]"

Using version ^0.12.2 for unstructured

Updating dependencies
Resolving dependencies... (30.2s)://files.pythonhosted.org/packages/56/fc/a3c13ded7b3057680c8ae95a9b6cc83e63657c38e0005c400a5d018a33a7/pyreadline3-3.4.1-py3-none-any.whl (23.5s)m9;2m(14.9s)Resolving dependencies... Downloading https://files.pythonhosted.org/packages/5e/55/7a85d92b115bb530797cd2ac5c11b905acbfd1d71cea88207eb276d1506a/pypdfium2-4.26.0-py3-none-macosx_10_13_x86_64.whl  22% (10.8s)Resolving dependencies... (16.4s)Resolving dependencies... (17.1s)Resolving dependencies... (19.1s)

Package operations: 57 installs, 0 updates, 0 removals

  • Installing mpmath (1.3.0): Pending...
  • Installing mpmath (1.3.0): Installing...
  • Installing mpmath (1.3.0)
  • Installing filelock (3.13.1): Pending...
  • Installing fsspec (2023.12.2): Pending...
  • Installing networkx (3.2.1): Pending...
  • Installing sympy (1.12): Pending...
  • Installing networkx (3.2.1): Pending...
  • Installing sympy (1.12): Pending...
  • Insta

In [ ]:
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader('./', glob="**/*.md")
docs = loader.load()

#### 加载CSV

#### 加载HTML

In [5]:
!poetry add parser


Could not find a matching version of package parser


In [14]:
!poetry add nest_asyncio

Using version ^1.6.0 for nest-asyncio

Updating dependencies
Resolving dependencies... (10.4s)Resolving dependencies... (7.4s)Resolving dependencies... (7.6s)

No dependencies to install or update

Writing lock file


In [16]:
# 仅在jupyter中需要
import nest_asyncio
nest_asyncio.apply()

In [37]:
from bs4 import BeautifulSoup, SoupStrainer
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader
from langchain_community.document_loaders.sitemap import SitemapLoader
from langchain_core.utils.html import PREFIXES_TO_IGNORE_REGEX, SUFFIXES_TO_IGNORE_REGEX
from parser import langchain_docs_extractor
import re

In [38]:
# 提取langchain的Docs文档
def metadata_extractor(meta: dict, soup: BeautifulSoup) -> dict:
    title = soup.find("title")
    description = soup.find("meta", attrs={"name": "description"})
    html = soup.find("html")
    return {
        "source": meta["loc"],
        "title": title.get_text() if title else "",
        "description": description.get("content", "") if description else "",
        "language": html.get("lang", "") if html else "",
        **meta,
    }

def load_langchain_docs():
    return SitemapLoader(
        "https://python.langchain.com/sitemap.xml",
        filter_urls=["https://python.langchain.com/"],
        parsing_function=langchain_docs_extractor,
        default_parser="lxml",
        bs_kwargs={
            "parse_only": SoupStrainer(
                name=("article", "title", "html", "lang", "content")
            ),
        },
        meta_function=metadata_extractor,
    ).load()

In [39]:
# 提取langchain的API文档
def simple_extractor(html: str) -> str:
    soup = BeautifulSoup(html, "lxml")
    return re.sub(r"\n\n+", "\n\n", soup.text).strip()

def load_api_docs():
    return RecursiveUrlLoader(
        url="https://api.python.langchain.com/en/latest/",
        max_depth=8,
        extractor=simple_extractor,
        prevent_outside=True,
        use_async=True,
        timeout=600,
        # Drop trailing / to avoid duplicate pages.
        link_regex=(
            f"href=[\"']{PREFIXES_TO_IGNORE_REGEX}((?:{SUFFIXES_TO_IGNORE_REGEX}.)*?)"
            r"(?:[\#'\"]|\/[\#'\"])"
        ),
        check_response_status=True,
        exclude_dirs=(
            "https://api.python.langchain.com/en/latest/_sources",
            "https://api.python.langchain.com/en/latest/_modules",
        ),
    ).load()

In [40]:
# langsmith的docs文档
def load_langsmith_docs():
    return RecursiveUrlLoader(
        url="https://docs.smith.langchain.com/",
        max_depth=8,
        extractor=simple_extractor,
        prevent_outside=True,
        use_async=True,
        timeout=600,
        # Drop trailing / to avoid duplicate pages.
        link_regex=(
            f"href=[\"']{PREFIXES_TO_IGNORE_REGEX}((?:{SUFFIXES_TO_IGNORE_REGEX}.)*?)"
            r"(?:[\#'\"]|\/[\#'\"])"
        ),
        check_response_status=True,
    ).load()

In [ ]:
langchain_docs = await load_langchain_docs()

In [ ]:
langchain_api = await load_api_docs()

In [ ]:
langchain_api

#### 加载JSON

#### 加载Markdown

In [152]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./index.md")
loader.load()

[Document(page_content='ok', metadata={'source': './index.md'})]

#### 加载PDF

### 文本切分

#### HTMLHeaderTextSplitter

In [2]:
from langchain.text_splitter import HTMLHeaderTextSplitter

html_string = """
<!DOCTYPE html>
<html>
<body>
    <div>
        <h1>Foo</h1>
        <p>Some intro text about Foo.</p>
        <div>
            <h2>Bar main section</h2>
            <p>Some intro text about Bar.</p>
            <h3>Bar subsection 1</h3>
            <p>Some text about the first subtopic of Bar.</p>
            <h3>Bar subsection 2</h3>
            <p>Some text about the second subtopic of Bar.</p>
        </div>
        <div>
            <h2>Baz</h2>
            <p>Some text about Baz</p>
        </div>
        <br>
        <p>Some concluding text about Foo</p>
    </div>
</body>
</html>
"""

headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
html_header_splits = html_splitter.split_text(html_string)
html_header_splits

[Document(page_content='Foo'),
 Document(page_content='Some intro text about Foo.  \nBar main section Bar subsection 1 Bar subsection 2', metadata={'Header 1': 'Foo'}),
 Document(page_content='Some intro text about Bar.', metadata={'Header 1': 'Foo', 'Header 2': 'Bar main section'}),
 Document(page_content='Some text about the first subtopic of Bar.', metadata={'Header 1': 'Foo', 'Header 2': 'Bar main section', 'Header 3': 'Bar subsection 1'}),
 Document(page_content='Some text about the second subtopic of Bar.', metadata={'Header 1': 'Foo', 'Header 2': 'Bar main section', 'Header 3': 'Bar subsection 2'}),
 Document(page_content='Baz', metadata={'Header 1': 'Foo'}),
 Document(page_content='Some text about Baz', metadata={'Header 1': 'Foo', 'Header 2': 'Baz'}),
 Document(page_content='Some concluding text about Foo', metadata={'Header 1': 'Foo'})]

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

url = "http://www.hongmeng-info.com/"

headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
    ("h4", "Header 4"),
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# for local file use html_splitter.split_text_from_file(<path_to_file>)
html_header_splits = html_splitter.split_text_from_url(url)

chunk_size = 500
chunk_overlap = 30
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

# Split
splits = text_splitter.split_documents(html_header_splits)
splits[:10]

[Document(page_content='Toggle navigation  \n首页 互联网应用 信息化服务 电子税务 招聘 联系我们  \nPrevious Next  \n稳健、高效 人性化的电子竞价系统, 千亿级股权交易平台实践检验'),
 Document(page_content='鸿蒙在线竞价系统，微信，APP，智能终端，多媒体控制.  \n了解更多>>', metadata={'Header 2': '稳健、高效 人性化的电子竞价系统, 千亿级股权交易平台实践检验'}),
 Document(page_content='会员平台及CRM管理'),
 Document(page_content='社群运营的基础架构系统，支持复杂权益，积分管理，“会员卡”系统，权益/积分商城应用，社群用户关系管理，多种智能行为数据模型，面向客户群/社群运营者提供有效的解决方案.  \n详情 »', metadata={'Header 2': '会员平台及CRM管理'}),
 Document(page_content='互联网运营平台'),
 Document(page_content='核心组件系统支撑O2O类运营体系，订单系统，合作商/供应商/渠道商管理及结算系统，客服系统，营销支撑与分析，活动及传播系统，为运营提供有效灵活的支撑.  \n详情 »', metadata={'Header 2': '互联网运营平台'}),
 Document(page_content='电子商城'),
 Document(page_content='服务/产品的线上交易平台。根据你的需要，实现你想要的电子商城系统。模块化组合实现满足不同运营方多层次的系统需要.  \n详情 »', metadata={'Header 2': '电子商城'}),
 Document(page_content='数据技术&服务'),
 Document(page_content='清洗，分析，挖掘，分析。经验和自有的工具务实有效的解决深层次运营问题。我们擅长解决各种类型的数据接口.  \n详情 »', metadata={'Header 2': '数据技术&服务'})]

### 向量编码

In [5]:
from langchain_openai import OpenAIEmbeddings
embeddings_model = OpenAIEmbeddings()

In [6]:
embeddings = embeddings_model.embed_documents(
    [
        "Hi there!",
        "Oh, hello!",
        "What's your name?",
        "My friends call me World",
        "Hello World!"
    ]
)
len(embeddings), len(embeddings[0])

(5, 1536)

In [7]:
embedded_query = embeddings_model.embed_query("对话中提及的名字是什么?")
embedded_query[:5]

[-0.0037282993122298882,
 -0.01327033122820681,
 0.03234768953778434,
 0.0035204264887496776,
 -0.017729538998712827]

**CacheBackedEmbeddings**：支持缓存

### 向量存储

#### 资料准备

In [17]:
!poetry add importlib

Using version ^1.0.4 for importlib

Updating dependencies
Resolving dependencies... (28.8s)://files.pythonhosted.org/packages/31/77/3781f65cafe55480b56914def99022a5d2965a4bb269655c89ef2f1de3cd/importlib-1.0.4.zip (0.6s)Resolving dependencies... (6.9s)Resolving dependencies... (10.3s)Resolving dependencies... (20.0s)

Package operations: 1 install, 0 updates, 0 removals

  • Installing importlib (1.0.4): Pending...
  • Installing importlib (1.0.4): Downloading... 0%
  • Installing importlib (1.0.4): Downloading... 100%
  • Installing importlib (1.0.4): Preparing...
  • Installing importlib (1.0.4): Installing...
  • Installing importlib (1.0.4)

Writing lock file


In [30]:
import os
import importlib.util
spec = importlib.util.find_spec('langchain')
langchain_files_path = os.path.join(os.path.dirname(spec.origin), "docs/docs/modules")
print(langchain_files_path)

/Users/xuehongwei/Library/Caches/pypoetry/virtualenvs/md-8WLN4Vov-py3.10/lib/python3.10/site-packages/langchain/docs/docs/modules


#### Chroma

In [8]:
!poetry add chromadb

Using version ^0.4.22 for chromadb

Updating dependencies
Resolving dependencies... (63.2s)://files.pythonhosted.org/packages/79/4d/9cc401e7b07e80532ebc8c8e993f42541534da9e9249c59ee0139dcb0352/websockets-12.0-py3-none-any.whl (55.6s)39m (36.2s)

Package operations: 41 installs, 1 update, 0 removals

  • Installing zipp (3.17.0): Pending...
  • Installing zipp (3.17.0): Installing...
  • Installing zipp (3.17.0)
  • Installing importlib-metadata (6.11.0): Pending...
  • Installing importlib-metadata (6.11.0): Downloading... 0%
  • Installing importlib-metadata (6.11.0): Downloading... 100%
  • Installing importlib-metadata (6.11.0): Installing...
  • Installing importlib-metadata (6.11.0)
  • Installing opentelemetry-api (1.22.0): Pending...
  • Installing pyasn1 (0.5.1): Pending...
  • Updating setuptools (69.0.2 -> 69.0.3): Pending...
  • Installing pyasn1 (0.5.1): Pending...
  • Updating setuptools (69.0.2 -> 69.0.3): Pending...
  • Installing opentelemetry-api (1.22.0): Downloading.

In [37]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Chroma

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
file = 'state_of_the_union.txt'
print(file)
raw_documents = TextLoader(file).load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)
db = Chroma.from_documents(documents, OpenAIEmbeddings())

state_of_the_union.txt


用**字符串参数**做相似性查询

In [39]:
query = "总统关于 Ketanji Brown Jackson 的发言"
docs = db.similarity_search(query)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


用**向量参数**做相似性查询（可以做更深度优化，例如减少向量编码的事件）

In [40]:
embedding_vector = OpenAIEmbeddings().embed_query(query)
docs = db.similarity_search_by_vector(embedding_vector)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


#### FAISS

In [72]:
!poetry add faiss-cpu

Using version ^1.7.4 for faiss-cpu

Updating dependencies
Resolving dependencies... (12.4s)://files.pythonhosted.org/packages/e4/5d/c35f5285b85b54b4b154ce40a8810d57a306f2da4a9a58cb7498f9aefadb/faiss_cpu-1.7.4-cp310-cp310-macosx_10_9_x86_64.whl  99% (0.5s)Resolving dependencies... (3.3s)Resolving dependencies... (4.5s)

Package operations: 1 install, 0 updates, 0 removals

  • Installing faiss-cpu (1.7.4): Pending...
  • Installing faiss-cpu (1.7.4): Downloading... 0%
  • Installing faiss-cpu (1.7.4): Downloading... 30%
  • Installing faiss-cpu (1.7.4): Downloading... 48%
  • Installing faiss-cpu (1.7.4): Downloading... 60%
  • Installing faiss-cpu (1.7.4): Downloading... 90%
  • Installing faiss-cpu (1.7.4): Downloading... 100%
  • Installing faiss-cpu (1.7.4): Installing...
  • Installing faiss-cpu (1.7.4)

Writing lock file


#### lanceDB

In [42]:
!poetry add lancedb

Using version ^0.5.1 for lancedb

Updating dependencies
Resolving dependencies... (16.8s)://files.pythonhosted.org/packages/f6/f0/10642828a8dfb741e5f3fbaac830550a518a775c7fff6f04a007259b0548/py-1.11.0-py2.py3-none-any.whl (3.1s)Resolving dependencies... (8.0s)Resolving dependencies... (8.4s)Resolving dependencies... (9.8s)Resolving dependencies... (11.0s)Resolving dependencies... (12.0s)Resolving dependencies... (12.6s)Resolving dependencies... (13.7s)Resolving dependencies... (16.5s)

Package operations: 8 installs, 0 updates, 0 removals

  • Installing py (1.11.0): Pending...
  • Installing pyarrow (15.0.0): Pending...
  • Installing pyarrow (15.0.0): Downloading... 0%
  • Installing pyarrow (15.0.0): Downloading... 0%
  • Installing py (1.11.0): Downloading... 0%
  • Installing pyarrow (15.0.0): Downloading... 0%
  • Installing pyarrow (15.0.0): Downloading... 0%
  • Installing py (1.11.0): Downloading... 100%
  • Installing pyarrow (15.0.0): Downloading... 0%
  • Installing pyarrow

In [44]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import LanceDB

embeddings = OpenAIEmbeddings()

import lancedb
db = lancedb.connect("/tmp/lancedb")

table = db.create_table(
    "my_table",
    data=[
        {
            "vector": embeddings.embed_query("Hello World"),
            "text": "Hello World",
            "id": "1",
        }
    ],
    mode="overwrite",
)

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
raw_documents = TextLoader('state_of_the_union.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)
db = LanceDB.from_documents(documents, OpenAIEmbeddings(), connection=table)

[2024-01-24T08:00:55Z WARN  lance::dataset] No existing dataset at /tmp/lancedb/my_table.lance, it will be created


使用**字符串**做相似性检索

In [48]:
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


使用**向量**做相似性检索

In [50]:
retriever = db.as_retriever()

### 向量检索

#### RAG

In [52]:

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

template = """Answer the question based only on the following context:

{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()


def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke("总统发言提到了什么技术方面的内容?")


'总统发言提到了新兴技术和美国制造业方面的内容。'

#### 定制Recevier

In [53]:
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.documents import Document
from typing import List

class CustomRetriever(BaseRetriever):    
    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        return [Document(page_content=query)]

retriever = CustomRetriever()

retriever.get_relevant_documents("bar")

[Document(page_content='bar')]

## 链式调用

可以直接使用 **LCEL** 语法构建自己的链，也可以使用现成的。
使用前最好直接查看 **langchain** 源代码。
- 使用 OpenAI function calling
- 创建数据库查询
- 检索文档
- ...

In [ ]:
from langchain_community.chat_models import ChatOpenAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain import hub

retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")
llm = ChatOpenAI()
retriever = ...
combine_docs_chain = create_stuff_documents_chain(
    llm, retrieval_qa_chat_prompt
)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

chain.invoke({"input": "..."})

## 记忆封装

## 智能体

### 简单的智能体例子

#### 工具1：搜索

In [55]:
import os
import getpass

os.environ['TAVILY_API_KEY'] = getpass.getpass('TAVILY API Key:')

TAVILY API Key: ········


##### 定义查询工具

In [92]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults()

In [93]:
search.invoke({"query": "最火的修仙小说"})

[{'url': 'https://www.52shuku.vip/Top/XiuXian.html',
  'content': '2024年修仙小说排行榜Top200，都是完结好文 小五根据用户真实点击为您整理出2024年修仙小说排行榜，您可以在线阅读文笔好，质量高，剧情在线的修仙小说，让你不再书荒。  [穿越重生] 《穿书之炮灰也要去修仙》作者：霍小苗【完结】 文案  《穿书之女配修仙纪》作者：凤羽零落【完结】\u3000\u3000文案：\u3000\u3000末世爆发后不久，死去的古月借尸还魂了，还魂到“仙珠”一书中的小配角身上，成为了书中第一个被女主踩下…… （获赞：19966 ） 38、[穿越重生耽美] 药仙_静舟小妖【完结】  [穿越重生] 《师傅就要黑化了》作者：沧娆【完结+番外】 【文案一】 夏微澜穿书了，穿到了一本修仙小说，不仅倒霉穿成了和女主抢男人的炮灰女配，还穿成了书中反派的徒弟。 多年前，谪仙师傅牵着她的小手,眉眼温柔，白衣飘飘，仙气十足。1、[穿越重生耽美] 穿越之修仙_衣落成火【完结】 · 2、[言情小说] 修仙女配要上天_脑壳有包【完结】 · 3、[言情小说] 从修士到寡妇[七十年代]_大河东流【完结+番外】 · 4、[\xa0...'},
 {'url': 'https://www.sohu.com/a/752544587_121888110',
  'content': '2024 十一本已完结仙侠类网络小说重磅推荐 有看过的吗？ 原标题：2024 十一本已完结仙侠类网络小说重磅推荐 有看过的吗？ 大家好，又到了今天的推书环节，看得好的书记得关注点赞收藏。废话不多说，下面进入正题。 《我为长生仙》  如果你喜欢修仙类的小说，或者对奇幻和冒险元素感兴趣，那么这部小说可能值得一试。但如果你对仙侠小说的要求非常高，或者对语言的流畅性和情节的逻辑性非常挑剔，那么可能需要谨慎考虑是否要阅读这部小说。 《大奉打更人》 作者：卖报小郎君  《苟在妖武乱世修仙》 作者：文抄公 字数：354.34万字 完结状态：已完结 内容梗概：  小说优点： 1.新颖的题材融合：将探案与修仙两大题材巧妙结合，创造出一个独特的叙事风格，令人耳目一新。 2.丰富的剧情层次：故事中融入了悬疑、奇幻、武侠等多种元素，使得剧情层次丰富，引人入胜。7 days ago —

##### 集成到chain

In [65]:
!poetry add langchain-openai langchainhub

The following packages are already present in the pyproject.toml and will be skipped:

  • langchain-openai

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Using version ^0.1.14 for langchainhub

Updating dependencies
Resolving dependencies... (18.2s)Resolving dependencies... (15.7s)

Package operations: 2 installs, 0 updates, 0 removals

  • Installing types-requests (2.31.0.20240106): Pending...
  • Installing types-requests (2.31.0.20240106): Downloading... 0%
  • Installing types-requests (2.31.0.20240106): Downloading... 100%
  • Installing types-requests (2.31.0.20240106): Installing...
  • Installing types-requests (2.31.0.20240106)
  • Installing langchainhub (0.1.14): Pending...
  • Installing langchainhub (0.1.14): Downloading... 0%
  • Installing langchainhub (0.1.14): Downloading... 100%
  • Installing langchainhub (0.1.14):

In [66]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_openai import ChatOpenAI

instructions = """You are an assistant."""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)
llm = ChatOpenAI(temperature=0)
tavily_tool = TavilySearchResults()
tools = [tavily_tool]
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)

In [68]:
agent_executor.invoke({"input": "推荐3部最火的都市灵异修仙小说"})



> Entering new AgentExecutor chain...
以下是3部最火的都市灵异修仙小说的推荐：

1. 《都市修仙高手》：这是一部非常受欢迎的都市灵异修仙小说，讲述了主人公在都市中修炼成为修仙高手的故事。小说中融合了都市生活和神秘灵异的元素，情节紧凑，引人入胜。

2. 《都市封神榜》：这是一部以都市为背景的灵异修仙小说，讲述了主人公在都市中修炼成为封神榜上的强者的故事。小说中充满了惊险刺激的情节和精彩的战斗场面，深受读者喜爱。

3. 《都市至尊仙尊》：这是一部热门的都市灵异修仙小说，讲述了主人公在都市中修炼成为至尊仙尊的故事。小说中融合了都市生活和仙侠修真的元素，情节扣人心弦，引人入胜。

这些小说都具有精彩的情节和丰富的修仙元素，非常适合喜欢都市灵异修仙题材的读者阅读。

> Finished chain.


{'input': '推荐3部最火的都市灵异修仙小说',
 'output': '以下是3部最火的都市灵异修仙小说的推荐：\n\n1. 《都市修仙高手》：这是一部非常受欢迎的都市灵异修仙小说，讲述了主人公在都市中修炼成为修仙高手的故事。小说中融合了都市生活和神秘灵异的元素，情节紧凑，引人入胜。\n\n2. 《都市封神榜》：这是一部以都市为背景的灵异修仙小说，讲述了主人公在都市中修炼成为封神榜上的强者的故事。小说中充满了惊险刺激的情节和精彩的战斗场面，深受读者喜爱。\n\n3. 《都市至尊仙尊》：这是一部热门的都市灵异修仙小说，讲述了主人公在都市中修炼成为至尊仙尊的故事。小说中融合了都市生活和仙侠修真的元素，情节扣人心弦，引人入胜。\n\n这些小说都具有精彩的情节和丰富的修仙元素，非常适合喜欢都市灵异修仙题材的读者阅读。'}

#### 工具2: 向量文本检索

In [94]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [95]:
retriever.get_relevant_documents("如何上传数据集")[0]

Document(page_content="dataset uploading.Once we have a dataset, how can we use it to test changes to a prompt or chain? The most basic approach is to run the chain over the data points and visualize the outputs. Despite technological advancements, there still is no substitute for looking at outputs by eye. Currently, running the chain over the data points needs to be done client-side. The LangSmith client makes it easy to pull down a dataset and then run a chain over them, logging the results to a new project associated with the dataset. From there, you can review them. We've made it easy to assign feedback to runs and mark them as correct or incorrect directly in the web app, displaying aggregate statistics for each test project.We also make it easier to evaluate these runs. To that end, we've added a set of evaluators to the open-source LangChain library. These evaluators can be specified when initiating a test run and will evaluate the results once the test run completes. If we‚Äôr

In [96]:
from langchain.tools.retriever import create_retriever_tool

In [97]:
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [98]:
retriever_tool

Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x127d64280>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x128cf3310>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x127d64550>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x128cf3310>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'))

#### 定义工具

In [99]:
tools = [search, retriever_tool]

#### 创建智能体

In [100]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [106]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
print(prompt)
prompt.messages

input_variables=['agent_scratchpad', 'input'] input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(variable_name='agent_

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [102]:
from langchain.agents import create_openai_functions_agent
agent = create_openai_functions_agent(llm, tools, prompt)

In [103]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

#### 运行智能体

In [88]:
agent_executor.invoke({"input": "hi!"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In [89]:
agent_executor.invoke({"input": "怎样使用angsmith帮助做测试呢？"})



> Entering new AgentExecutor chain...
要使用Angsmith来帮助进行测试，您可以按照以下步骤进行操作：

1. 安装Angsmith：首先，您需要安装Angsmith测试框架。您可以通过在终端中运行以下命令来安装Angsmith：

   ```
   npm install -g angsmith
   ```

2. 创建测试文件：在您的项目中创建一个新的测试文件，例如`test.js`。

3. 导入Angsmith：在测试文件的顶部，导入Angsmith模块：

   ```javascript
   const Angsmith = require('angsmith');
   ```

4. 创建测试套件：使用`Angsmith`对象创建一个新的测试套件：

   ```javascript
   const suite = new Angsmith.Suite('My Test Suite');
   ```

5. 添加测试用例：使用`suite.addTest`方法添加测试用例。测试用例由一个描述和一个测试函数组成：

   ```javascript
   suite.addTest('My Test Case', () => {
     // 测试逻辑
   });
   ```

6. 运行测试套件：使用`suite.run`方法运行测试套件，并在控制台中输出测试结果：

   ```javascript
   suite.run();
   ```

7. 运行测试文件：在终端中运行测试文件，使用以下命令：

   ```
   node test.js
   ```

   您将看到测试结果的输出，包括每个测试用例的状态（通过、失败或挂起）以及任何错误消息。

这样，您就可以使用Angsmith来帮助进行测试了。您可以根据需要添加更多的测试用例和测试套件，并使用Angsmith的其他功能来进行更复杂的测试。

> Finished chain.


{'input': '怎样使用angsmith帮助做测试呢？',
 'output': "要使用Angsmith来帮助进行测试，您可以按照以下步骤进行操作：\n\n1. 安装Angsmith：首先，您需要安装Angsmith测试框架。您可以通过在终端中运行以下命令来安装Angsmith：\n\n   ```\n   npm install -g angsmith\n   ```\n\n2. 创建测试文件：在您的项目中创建一个新的测试文件，例如`test.js`。\n\n3. 导入Angsmith：在测试文件的顶部，导入Angsmith模块：\n\n   ```javascript\n   const Angsmith = require('angsmith');\n   ```\n\n4. 创建测试套件：使用`Angsmith`对象创建一个新的测试套件：\n\n   ```javascript\n   const suite = new Angsmith.Suite('My Test Suite');\n   ```\n\n5. 添加测试用例：使用`suite.addTest`方法添加测试用例。测试用例由一个描述和一个测试函数组成：\n\n   ```javascript\n   suite.addTest('My Test Case', () => {\n     // 测试逻辑\n   });\n   ```\n\n6. 运行测试套件：使用`suite.run`方法运行测试套件，并在控制台中输出测试结果：\n\n   ```javascript\n   suite.run();\n   ```\n\n7. 运行测试文件：在终端中运行测试文件，使用以下命令：\n\n   ```\n   node test.js\n   ```\n\n   您将看到测试结果的输出，包括每个测试用例的状态（通过、失败或挂起）以及任何错误消息。\n\n这样，您就可以使用Angsmith来帮助进行测试了。您可以根据需要添加更多的测试用例和测试套件，并使用Angsmith的其他功能来进行更复杂的测试。"}

In [105]:
agent_executor.invoke({"input": "有关于openai的最新消息吗?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'latest news about OpenAI'}`


[{'url': 'https://www.nytimes.com/2024/01/08/technology/ai-robots-chatbots-2024.html', 'content': 'company OpenAI, was asked what surprises the field would bring in 2024.  Supported by Robots Learn, Chatbots Visualize: How 2024 Will Be A.I.’s ‘Leap Forward’  (The New York Times sued OpenAI and Microsoft last month for copyright infringement of news content related to A.I.  A.I. is set to advance at a rapid rate, becoming more powerful and spreading into the physical world. By Cade MetzAt an event in San Francisco in November, Sam Altman, the chief executive of the artificial intelligence company OpenAI, was asked what surprises the field would bring in 2024. Online chatbots...'}, {'url': 'https://www.trendforce.com/news/2024/01/24/news-openai-reportedly-expected-to-gather-with-samsung-and-sk-group-for-deepened-chip-collaboration/', 'content': 'Read more (Photo 

{'input': '有关于openai的最新消息吗?',
 'output': '以下是关于OpenAI的最新消息：\n\n1. 根据《纽约时报》的报道，OpenAI的首席执行官Sam Altman在11月的一个活动中被问及人工智能领域在2024年会带来哪些惊喜。他表示，人工智能将以快速的速度发展，变得更加强大，并扩展到物理世界中。此外，OpenAI和微软最近因涉嫌侵犯与人工智能相关的新闻内容的版权而被《纽约时报》起诉。\n\n2. 据《趋势力》报道，OpenAI的首席执行官Sam Altman计划于1月26日访问韩国。他预计将与三星电子和SK集团的高层举行会议，加强在高带宽内存（HBM）领域的合作。\n\n3. 《技术评论》的文章指出，2024年人工智能的四大热门趋势包括OpenAI传闻中的新型Q*模型、生成式人工智能在非技术人员中的实际应用以及人们对各种人工智能模型的探索和研究。\n\n4. 根据《大西洋月刊》的报道，全球正在努力重新定义人工智能的“开源”概念，以限制人们对OpenAI等公司模型的研究、复制或竞争。\n\n5. 根据彭博社的报道，OpenAI的首席执行官Sam Altman表示，OpenAI的领导变动对他来说比让人工智能达到人类水平的压力更轻松。\n\n请注意，以上信息仅为搜索结果摘要，具体内容请点击链接查看详细报道。'}

### 概念

#### Schema

LangChain定义了让更容易让智能体工作的几个抽象类。

##### AgentAction

这是包装数据的类，包含一个**tool**属性和一个**tool_input**属性。

##### AgentFinish

返回最终结果。

##### Intermediate Steps

当前智能体返回的中间结果，会作为将来继续执行时的上下文。

#### Agent

这是一个决定下一步如何执行的 **chain**，通常会包含大模型、提示语和输出解析等关键部件。

##### Agent Inputs

通常是包含 **Intermediate Steps** 的键值对。

##### Agent Outputs

通常是 **AgentAction** 或 **AgentFinish** 。

#### AgentExecutor

调度 **Agent** 运行时，选择并执行其中包含的 **actions**。

下面是简单的例子：

In [ ]:
next_action = agent.get_action(...)
while next_action != AgentFinish:
    observation = run(next_action)
    next_action = agent.get_action(..., next_action, observation)
return next_action

你还应当考虑：

- 处理选中的 **tool** 不存在的情况
- 处理 **tool** 中的异常
- 处理智能体的输出结果无法映射到 **tool** 调用的情况
- 打印或结合 **langfuse/langsmith** 处理日志和各层级的调试信息

#### Tools

**Tool** 的抽象是告诉大模型哪些参数可以被调用。包含两个部件。

- 参数描述：如果没有这部份，大模型就无法确定该输入什么参数。这些参数应当是被良好命名和描述的。
- 执行函数：一般是 **Python** 调用。

##### 注意思考

- 必须为智能体提供可用的 **Tool**
- 必须为智能体提供良好的描述

#### Toolkits

**langchain** 为很多通用任务准备了开箱即用的工具集，如：
- 生成或执行 **python** 代码 [https://python.langchain.com/docs/integrations/toolkits/python]
- 浏览器机器人 [https://python.langchain.com/docs/integrations/toolkits/playwright]
- 比较两个文档 [https://python.langchain.com/docs/integrations/toolkits/document_comparison_toolkit]
- 处理 **CSV** [https://python.langchain.com/docs/integrations/toolkits/csv]
- 处理 **pandas** [https://python.langchain.com/docs/integrations/toolkits/pandas]
- 处理 **SQL** [https://python.langchain.com/docs/integrations/toolkits/sql_database]
- 访问 **github** [https://python.langchain.com/docs/integrations/toolkits/github]
- 访问 **gitlab** [https://python.langchain.com/docs/integrations/toolkits/gitlab]
- 访问 **PowerBI** [https://python.langchain.com/docs/integrations/toolkits/powerbi]

### 智能体类型

智能体可以按这些维度分类：
- 预期的模型是对话模型还是单次模型
- 是否支持多参数输入的工具
- 是否支持工具的并发调度
- 是否需要模型的附加参数

当前**langchain**主要包括的智能体类型有：

- **ReAct Agent**：是一个简单的Agent，适用于简单的模型。它使用ReAct框架来确定使用哪个工具，并且不支持多输入工具。适用于简单的问题和任务。
- **Self Ask With Search Agent**：是一个简单的Agent，适用于简单的模型和只有一个搜索工具的情况。它将复杂的问题分解为一系列简单的问题，并使用搜索工具查找答案。适用于需要使用搜索工具查找答案的问题。
- **OpenAI Tools Agent**：是一个适用于Chat模型的Agent，支持聊天历史和多输入工具。它可以与最新的OpenAI模型一起使用，并且可以调用多个工具并行执行任务。适用于需要使用多个工具并且需要聊天历史的任务。
- **OpenAI Functions Agent**：是一个适用于Chat模型的Agent，支持聊天历史和多输入工具。它专门针对使用OpenAI函数的模型进行了优化，并且可以调用多个函数并行执行任务。适用于使用OpenAI模型或经过微调以支持函数调用的开源模型的任务。
- **XML Agent**：是一个使用XML标记的Agent，适用于处理XML格式的工具输入和输出。适用于需要处理XML数据的任务。
- **JSON Agent**：是一个使用JSON格式的Agent，适用于处理JSON格式的工具输入和输出。适用于需要处理JSON数据的任务。
- **Structured Chat Agent**：是一个适用于Chat模型的Agent，支持聊天历史和多输入工具。它可以处理具有多个输入的工具，并支持更复杂的工具使用场景，如精确导航浏览器。适用于需要处理具有多个输入的工具的任务。

其中：**OpenAI Tools Agent** 需要与最新的OpenAI模型一起使用，因为它利用了OpenAI的最新功能，即并行函数调用（tool calling）。
这种功能允许模型一次性返回多个函数调用结果，从而提高了效率和性能。
因此，为了充分利用**OpenAI Tools Agent**的功能，需要使用支持并行函数调用的最新OpenAI模型。

### ReAct Agent 示例

#### 定义智能体

In [230]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import OpenAI

In [231]:
tools = [TavilySearchResults(max_results=5)]

In [232]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")

In [233]:
# Choose the LLM to use
llm = OpenAI()

# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)

#### 运行智能体

In [234]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [236]:
agent_executor.invoke({"input": "吴京的老婆是谁？"})



> Entering new AgentExecutor chain...
 You should always think about what search query would be the most effective for finding the answer.
Action: tavily_search_results_json
Action Input: 吴京的老婆[{'url': 'https://www.wenxuecity.com/news/2024/01/24/ent-253317.html', 'content': '另一方面，吴京又对现代婚姻有向往，因为他也想获得那种亲密且平等的夫妻感受，他也欣赏有能量有能力的女人，所以相比起其他纯纯的大男子主义，吴京又有一些不那么大男子主义的行为，会逗老婆开心，会在生活上照料她，也会逼迫自己做一些公开亲昵的举动。  吴京的确是一位很有意思的演员，无论事业还是婚姻，他都承载着中国广大男性的基本价值观；  到了2019年，吴京担任了《流浪地球》的出品人兼主演，事业再上新高度，那一年的福布斯名人榜，吴京是第一名。 吴京也乐于对外展示“宠妻”的形象，最著名的例子是和女明星合影从来没有越矩的行为，连胳膊都不碰，被网友称为“男德班课代表”。  2014年8月谢楠生下早产儿子，此后就基本上退圈，在家相夫教子。五个月后，吴京的《战狼》过审，再到2015年8月，终于上映，席卷票房，舆论火爆，吴京成功晋身为华语电影导演。 “窒息式”婚姻究竟从何而来？文章来源: 蓝小姐和黄小姐 于 2024-01-24 19:18:00 - 新闻取自各大新闻媒体，新闻内容并不代表本网立场 ... 对于老婆要不要工作这件事，客观来讲吴京是持开放态度的，他曾在接受李静采访时表达过，女孩子要有自己的事业，不要做家庭主妇，不然眼光变窄\xa0...'}, {'url': 'https://www.sohu.com/a/667852446_121621626', 'content': '就连吴京和谢楠的婚礼现场，作为新郎的吴京都是瘸着腿参加的。 在婚礼现场，拄着拐的吴京对新娘谢楠深情告白：“老婆，我以后再也不跳楼了。” 这也是吴京成为公认的“铁人”“硬汉”的原因之一。  由此可见，吴京的“硬汉”形象已经深入人心。 然而，就算是这样一位“硬汉”，

{'input': '吴京的老婆是谁？', 'output': 'Xie Nan'}

In [235]:
agent_executor.invoke({"input": "吴京的老婆主持过什么综艺？"})



> Entering new AgentExecutor chain...
 Think about possible keywords related to 吴京 and his wife
Action: tavily_search_results_json
Action Input: 吴京 老婆 综艺[{'url': 'https://www.wenxuecity.com/news/2024/01/24/ent-253317.html', 'content': '▲萌娃们。包括王诗龄和石头。 而整场婚礼都是怀着孕的谢楠一个人策划准备的，她自称是“总导演，策划，兼主演”，对吴京的唯一要求就是“你跟剧组请一天的假过来”。 吴京也坦言，结婚就像参加了一个综艺节目。  经由吴彬推荐，21岁的吴京被选中，成为了“第二个李连杰”，去香港演了自己的处女作《功夫小子闯情关》。  到了2019年，吴京担任了《流浪地球》的出品人兼主演，事业再上新高度，那一年的福布斯名人榜，吴京是第一名。 吴京也乐于对外展示“宠妻”的形象，最著名的例子是和女明星合影从来没有越矩的行为，连胳膊都不碰，被网友称为“男德班课代表”。  吴京的确是一位很有意思的演员，无论事业还是婚姻，他都承载着中国广大男性的基本价值观；6 hours ago — 6 hours ago吴京也坦言，结婚就像参加了一个综艺节目。 ... 就像谢楠，她也不知道该怎么去解决这个难题，但是，这个社会总是要往前走的，起码吴京最终也知道了“老婆已经\xa0...'}, {'url': 'https://www.backchina.com/news/2024/01/11/895312.html', 'content': '看得出来吴京是那种遇到比赛性质的事情就会永远想要赢的人。 但当众埋怨老婆，又真的大丈夫吗？ 气氛到这里，同样作为真人秀夫妻之一的郭京飞一句：“你的不对我觉得。” 郭京飞的一句客观的点评，就足以让谢楠湿了眼眶。  甚至吴京在两人正式确定前，不止一次的表示，爱她就要给她绝对自由的和工作空间。 并且积极表忠心，要把自己的卡和全部的财产交给亲亲老婆打理，只要给自己留给零花钱就行。 而婚后，吴京和谢楠的浪漫爱情依旧在生活细节处闪闪发光。  吴京家暴谢楠打到骨折？对她呼来喝去发大疯.... 京港台：2024-1-11 12:14

{'input': '吴京的老婆主持过什么综艺？',
 'output': "It seems that there are no results mentioning a variety show hosted by 吴京's wife."}

#### 增加历史消息支持

In [124]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react-chat")

In [125]:
# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [126]:
from langchain_core.messages import AIMessage, HumanMessage

agent_executor.invoke(
    {
        "input": "what's my name? Only use a tool if needed, otherwise respond with Final Answer",
        # Notice that chat_history is a string, since this prompt is aimed at LLMs, not chat models
        "chat_history": "Human: Hi! My name is Bob\nAI: Hello Bob! Nice to meet you",
    }
)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: Your name is Bob.

> Finished chain.


{'input': "what's my name? Only use a tool if needed, otherwise respond with Final Answer",
 'chat_history': 'Human: Hi! My name is Bob\nAI: Hello Bob! Nice to meet you',
 'output': 'Your name is Bob.'}

### OpenAI Tools Agent 示例

#### 定义智能体

In [127]:
!poetry add langchain-openai tavily-python

The following packages are already present in the pyproject.toml and will be skipped:

  • langchain-openai

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Using version ^0.3.1 for tavily-python

Updating dependencies
Resolving dependencies... (14.8s)

Package operations: 1 install, 0 updates, 0 removals

  • Installing tavily-python (0.3.1): Pending...
  • Installing tavily-python (0.3.1): Downloading... 0%
  • Installing tavily-python (0.3.1): Downloading... 100%
  • Installing tavily-python (0.3.1): Installing...
  • Installing tavily-python (0.3.1)

Writing lock file


In [128]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI

In [129]:
tools = [TavilySearchResults(max_results=2)]

In [130]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")

In [131]:
# Choose the LLM that will drive the agent
# Only certain models support this
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(llm, tools, prompt)

#### 执行智能体

In [132]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [133]:
agent_executor.invoke({"input": "what is LangChain?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'LangChain'}`


[{'url': 'https://slashdev.io/blog/the-ultimate-guide-to-langchain-in-2024', 'content': 'LangChain represents the cutting edge in language model technology, heralding a new era of artificial intelligence (AI)  LangChain is built on a foundation of complex algorithms and machine learning models that form its core architecture,  LangChain’s transformative role is evident in several key areas that define the next generation of AI communication.  LangChain a highly versatile tool for building AI solutions that can cater to a broad spectrum of conversational needs.1. Introduction to LangChain: The Future of Language Models / LangChain represents the cutting edge in language model technology, heralding a new era of artificial intelligence (AI) with remarkable conversational abilities.'}, {'url': 'https://walkingtree.tech/langchain-unleashing-the-full-potential-of-llms/', 'content':

{'input': 'what is LangChain?',
 'output': 'LangChain represents the cutting edge in language model technology, heralding a new era of artificial intelligence (AI) with remarkable conversational abilities. It is built on a foundation of complex algorithms and machine learning models that form its core architecture. LangChain is a highly versatile tool for building AI solutions that can cater to a broad spectrum of conversational needs. It provides the capability to interact with and query data from different sources and use it effectively. LangChain is a great framework that can be used for developing applications powered by LLMs (Large Language Models). It provides a structured and effective solution for leveraging the immense potential of LLMs to build astounding applications by providing a layer of abstraction around the LLMs and making their use easy and effective.\n\nFor more detailed information, you can visit the following links:\n1. [The Ultimate Guide to LangChain in 2024](htt

#### 支持消息历史

In [134]:
from langchain_core.messages import AIMessage, HumanMessage

agent_executor.invoke(
    {
        "input": "what's my name? Don't use tools to look this up unless you NEED to",
        "chat_history": [
            HumanMessage(content="hi! my name is bob"),
            AIMessage(content="Hello Bob! How can I assist you today?"),
        ],
    }
)



> Entering new AgentExecutor chain...
Your name is Bob.

> Finished chain.


{'input': "what's my name? Don't use tools to look this up unless you NEED to",
 'chat_history': [HumanMessage(content='hi! my name is bob'),
  AIMessage(content='Hello Bob! How can I assist you today?')],
 'output': 'Your name is Bob.'}

### Self-ask with search 示例

该Agent的实际效果似乎取决于大模型的能力，使用GPT4明显要优于GPT3.5版本。

In [243]:
from langchain import hub
from langchain.agents import AgentExecutor, create_self_ask_with_search_agent
from langchain_openai import OpenAI
from langchain_community.tools.tavily_search import TavilyAnswer

In [244]:
# 下载一个模板
prompt = hub.pull("hwchase17/self-ask-with-search")
print(prompt.template)

Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washin

In [250]:
# Choose the LLM that will drive the agent
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = "gpt-4-1106-preview")
# llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)

search = TavilyAnswer(max_results=3)

from langchain.agents import create_self_ask_with_search_agent
# tools = [
#     Tool(
#         name="Intermediate Answer",
#         func=search.run,
#         description="useful for when you need to ask with search.",
#     )
# ]
tools = [TavilyAnswer(max_results=1, name="Intermediate Answer")]
agent = create_self_ask_with_search_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [251]:
agent_executor.invoke({"input": "吴京的老婆主持过哪些综艺？"}, verbose = True, handle_parsing_errors=True)



> Entering new AgentExecutor chain...
Yes.
Follow up: Who is 吴京 (Wu Jing)'s wife?
Intermediate answer: 吴京 (Wu Jing)'s wife is Xie Nan.
Follow up: What variety shows has Xie Nan hosted?
Intermediate answer: Xie Nan has hosted shows such as "Happy Camp" (快乐大本营) and "Tiantian Xiangshang" (天天向上).
So the final answer is: Xie Nan has hosted variety shows like "Happy Camp" and "Tiantian Xiangshang".

> Finished chain.


{'input': '吴京的老婆主持过哪些综艺？',
 'output': 'Xie Nan has hosted variety shows like "Happy Camp" and "Tiantian Xiangshang".'}

In [225]:
agent_executor.invoke({"input": "马斯克的哪家公司最赚钱？"}, verbose = True)



> Entering new AgentExecutor chain...
Yes.
Follow up: Who is 马斯克?
Intermediate answer: 马斯克 (Mǎ sī kè) is Elon Musk in Mandarin Chinese.
Follow up: What companies does Elon Musk own?
Intermediate answer: Elon Musk is associated with several companies, including Tesla, Inc., SpaceX, Neuralink, and The Boring Company.
Follow up: Which of Elon Musk's companies is the most profitable?
Intermediate answer: As of my last update, Tesla, Inc. is considered the most profitable of Elon Musk's companies.
So the final answer is: Tesla, Inc. (特斯拉)

> Finished chain.


{'input': '马斯克的哪家公司最赚钱？', 'output': 'Tesla, Inc. (特斯拉)'}

### Tools

## 综合实践

### 搜索引擎

In [152]:
!poetry add duckduckgo-search

Using version ^4.3 for duckduckgo-search

Updating dependencies
Resolving dependencies... (20.5s)Resolving dependencies... (11.7s)Resolving dependencies... (14.6s)Resolving dependencies... (14.9s)Resolving dependencies... (15.0s)Resolving dependencies... (17.0s)Resolving dependencies... (17.1s)Resolving dependencies... (18.3s)Resolving dependencies... (20.6s)

Package operations: 3 installs, 0 updates, 0 removals

  • Installing curl-cffi (0.6.0b7): Pending...
  • Installing docstring-inheritance (2.1.2): Pending...
  • Installing docstring-inheritance (2.1.2): Downloading... 0%
  • Installing docstring-inheritance (2.1.2): Downloading... 100%
  • Installing docstring-inheritance (2.1.2): Installing...
  • Installing docstring-inheritance (2.1.2)
  • Installing docstring-inheritance (2.1.2)
  • Installing curl-cffi (0.6.0b7): Downloading... 0%
  • Installing docstring-inheritance (2.1.2)
  • Installing docstring-inheritance (2.1.2)
  • Installing curl-cffi (0.6.0b7): Downloading... 10%

In [161]:
from langchain.tools import DuckDuckGoSearchRun
import nest_asyncio
nest_asyncio.apply()

In [162]:
search = DuckDuckGoSearchRun()

In [165]:
search.run("Obama's first name?")

DuckDuckGoSearchException: _aget_url() https://duckduckgo.com RequestsError: Failed to perform, ErrCode: 28, Reason: 'Failed to connect to duckduckgo.com port 443 after 75010 ms: Couldn't connect to server'. This may be a libcurl error, See https://curl.se/libcurl/c/libcurl-errors.html first for more details.

### 通过文本向量路由Prompt

In [107]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

physics_template = """你是一个物理学教授，负责给数学爱好者解答疑惑。\
你正在为小学生回答问题，注意使用小学生水平能听懂的词汇，避免过于专业晦涩的术语。 \
当你不知道答案时就回答不知道。

Here is a question:
{query}"""

math_template = """你是一个数学家，负责给小学生解答疑惑。
注意使用小学生水平能听懂的词汇，避免过于专业晦涩的术语。 \
回答时，请举一些生活中的例子。
当你不知道答案时就回答不知道。

Here is a question:
{query}"""

embeddings = OpenAIEmbeddings()
prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)


def prompt_router(input):
    query_embedding = embeddings.embed_query(input["query"])
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    print("Using MATH" if most_similar == math_template else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)

chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | ChatOpenAI()
    | StrOutputParser()
)

common_train = ChatOpenAI() | StrOutputParser()

In [105]:
print(common_train.invoke("黑洞是什么？"))

黑洞是一种极度密集的天体，它具有非常强大的引力场，以至于连光都无法逃离它的吸引。黑洞的形成是由于一个恒星在死亡时，其质量过大，无法通过核聚变维持稳定，导致恒星坍缩成一个极为紧凑的物体。黑洞的中心部分称为奇点，奇点的密度和引力非常之大，超过了任何已知物质的极限。

黑洞的存在可以通过它们产生的引力效应来间接观测，例如吸收附近的物质、扭曲周围空间和发射强烈的辐射。虽然我们无法直接观测到黑洞，但科学家们通过观测它们对周围物体的影响，以及通过天文观测和数学模型来研究黑洞的性质和行为。

黑洞在宇宙中广泛存在，它们可能是恒星坍缩形成的中等质量黑洞，也可能是超大质量黑洞，如位于银河系中心的超大质量黑洞。黑洞对宇宙的演化和结构具有重要影响，它们是天体物理学和相对论研究的重要对象。


In [106]:
print(chain.invoke("黑洞是什么？"))

Using PHYSICS
小朋友，黑洞是宇宙中一种非常特殊的东西。它是一种非常强大的引力场，就像一个很大的吸力。当一颗非常大的恒星（就是我们看到的星星）燃烧完燃料后，它会塌缩成一个非常小又非常密集的东西，就是黑洞。黑洞的引力非常强大，甚至连光也无法逃脱它的吸引力。所以我们看不到黑洞，它是非常神秘的。关于黑洞，科学家们还在研究中，有很多有趣的发现等待我们去探索。


In [108]:
print(common_train.invoke("路径积分是什么？"))

路径积分是一个物理学概念，用来描述在一个力场中沿着一条曲线路径上的力的积累效果。简单来说，路径积分是将一个向量场沿着一条曲线进行积分，得到沿着该曲线的总体积效应。

在物理学中，路径积分可以用来计算沿着一个曲线路径上的力的总效果，比如沿着一条曲线上的力的总功或者总位移。路径积分的计算方式是将力场在曲线上的每个点上的力与微小位移相乘，然后将所有微小的力与位移的乘积相加，得到曲线上的总效果。

路径积分在许多领域中都有重要的应用，比如在力学中用于计算物体在曲线路径上的总功、在电磁学中用于计算电场或磁场沿着曲线的总位移等。路径积分的计算可以通过数学上的积分运算来实现，根据具体情况可以采用不同的积分方法，比如定积分或线积分等。


In [109]:
print(chain.invoke("路径积分是什么？"))

Using MATH
路径积分是一种数学工具，它在物理学中常常被用来描述粒子或光在空间中的运动。你可以把路径积分想象成一个粒子或光在不同路径上行走的概率，就像我们在城市里选择不同的路线去目的地一样。

想象一下你要从学校回家，有很多条路可以选择。每条路都有不同的长度、不同的交通状况和不同的风景。路径积分就是用来计算你选择每条路的概率，也就是说，你走每条路的可能性有多大。

在物理学中，粒子或光在空间中运动的时候，也有很多可能的路径可以选择。路径积分可以帮助我们计算出每条路径的概率，从而更好地理解粒子或光的行为。

但是，具体如何计算路径积分，需要更深入的数学知识和物理背景。这里只是简单介绍了路径积分的概念，如果你对它感兴趣，可以在以后的学习中深入了解。


### 执行python代码

In [110]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (
    ChatPromptTemplate,
)
from langchain_experimental.utilities import PythonREPL
from langchain_openai import ChatOpenAI

In [123]:
template = """Write some python code to solve the user's problem. 

Return only python code in Markdown format and Chinese, e.g.:

```python
....
```"""
prompt = ChatPromptTemplate.from_messages([("system", template), ("human", "{input}")])

model = ChatOpenAI()

In [124]:
def _sanitize_output(text: str):
    print(text)
    _, after = text.split("```python")
    return after.split("```")[0]

In [125]:
chain = prompt | model | StrOutputParser() | _sanitize_output | PythonREPL().run

In [126]:
chain.invoke({"input": "一个笼子里有兔子和鸡若干，数一数有5个头，12只脚，请问有多少只兔子多少只鸡？"})

我们可以使用穷举法来解决这个问题。

假设有 x 只兔子，y 只鸡。根据题意，可以得到以下两个方程：

x + y = 5   # 头的数量
4x + 2y = 12  # 脚的数量

我们可以通过求解这个方程组来得到兔子和鸡的数量。让我们来编写代码实现这个算法。

```python
def solve():
    for x in range(6):  # 兔子的数量最多为5只
        y = 5 - x  # 根据第一个方程计算鸡的数量
        if 4*x + 2*y == 12:  # 检查第二个方程是否满足
            return x, y  # 返回兔子和鸡的数量

rabbit, chicken = solve()
print(f"兔子的数量为：{rabbit} 只，鸡的数量为：{chicken} 只")
```

运行这段代码，我们可以得到输出：

```
兔子的数量为：1 只，鸡的数量为：4 只
```

所以，笼子里有1只兔子和4只鸡。


'兔子的数量为：1 只，鸡的数量为：4 只\n'

### 查询数据库

In [132]:
from langchain_core.prompts import ChatPromptTemplate

template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""
prompt = ChatPromptTemplate.from_template(template)

In [133]:
from langchain_community.utilities import SQLDatabase

In [134]:
db = SQLDatabase.from_uri("sqlite:///./Chinook.sqlite")

In [135]:
def get_schema(_):
    return db.get_table_info()

In [136]:
def run_query(query):
    return db.run(query)

In [137]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

sql_response = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | model.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

In [138]:
## 直接生成查询语句
sql_response.invoke({"question": "How many employees are there?"})

'SELECT COUNT(*) FROM Employee'

In [149]:
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}

请用中文回答。
"""
prompt_response = ChatPromptTemplate.from_template(template)

In [150]:
# 注意要分两阶段执行assign：先生成SQL，才能执行SQL
full_chain = (
    RunnablePassthrough.assign(
        schema=get_schema,
        query=sql_response
    ).assign(
        response=lambda x: db.run(x["query"]),
    )
    | prompt_response
    | model
)

In [151]:
full_chain.invoke({"question": "员工人数是多少?"})

AIMessage(content='员工人数是8人。')

### Stable Diffusion

In [91]:
from http import HTTPStatus
from urllib.parse import urlparse, unquote
from pathlib import PurePosixPath
import requests
import dashscope

model = "stable-diffusion-xl"
prompt = "Eagle flying freely in the blue sky and white clouds"


def simple_call():
    rsp = dashscope.ImageSynthesis.call(model=model,
                                        prompt=prompt,
                                        negative_prompt="garfield",
                                        n=1,
                                        size='1024*1024')
    if rsp.status_code == HTTPStatus.OK:
        print(rsp.output)
        print(rsp.usage)
        # save file to current directory
        for result in rsp.output.results:
            file_name = PurePosixPath(unquote(urlparse(result.url).path)).parts[-1]
            with open('./%s' % file_name, 'wb+') as f:
                f.write(requests.get(result.url).content)
    else:
        print('Failed, status_code: %s, code: %s, message: %s' %
              (rsp.status_code, rsp.code, rsp.message))


if __name__ == '__main__':
    simple_call()

{"task_id": "0203a963-42d2-400e-b025-f25cdbaec00e", "task_status": "SUCCEEDED", "results": [{"url": "https://dashscope-result-bj.oss-cn-beijing.aliyuncs.com/1d/db/20240127/7d5c308a/14cb51f7-ad6d-42b6-889a-18e516fb8b8f-1.png?Expires=1706452492&OSSAccessKeyId=LTAI5tQZd8AEcZX6KZV4G8qL&Signature=ZEmkERhxGOZYyIechN4Mphq2IOs%3D"}], "submit_time": "2024-01-27 22:34:35.041", "scheduled_time": "2024-01-27 22:34:35.063", "end_time": "2024-01-27 22:34:52.513", "task_metrics": {"TOTAL": 1, "SUCCEEDED": 1, "FAILED": 0}}
{"image_count": 1}


### 通义千问VL

In [94]:
from http import HTTPStatus
import dashscope


def simple_multimodal_conversation_call():
    """Simple single round multimodal conversation call.
    """
    messages = [
        {
            "role": "user",
            "content": [
                {"image": "https://dashscope.oss-cn-beijing.aliyuncs.com/images/dog_and_girl.jpeg"},
                {"text": "这是什么?"}
            ]
        }
    ]
    response = dashscope.MultiModalConversation.call(model='qwen-vl-plus',
                                                     messages=messages)
    # The response status_code is HTTPStatus.OK indicate success,
    # otherwise indicate request is failed, you can get error code
    # and message from code and message.
    if response.status_code == HTTPStatus.OK:
        print(response)
    else:
        print(response.code)  # The error code.
        print(response.message)  # The error message.


if __name__ == '__main__':
    simple_multimodal_conversation_call()

{"status_code": 200, "request_id": "5bffde55-34c3-949b-92ec-c1eb5e794a97", "code": "", "message": "", "output": {"text": null, "finish_reason": null, "choices": [{"finish_reason": "stop", "message": {"role": "assistant", "content": [{"text": "这张图片显示了一位女士和她的狗在海滩上。她们似乎正在享受彼此的陪伴，狗狗坐在沙滩上伸出爪子与女士握手或互动。背景是美丽的日落景色，海浪轻轻拍打着海岸线。\n\n请注意，我提供的描述基于图像中可见的内容，并不包括任何超出视觉信息之外的推测性解释。如果您需要更多关于场景、物体或其他细节的信息，请告诉我！"}]}}]}, "usage": {"input_tokens": 1277, "output_tokens": 85, "image_tokens": 1247}}


### 通义万相

In [93]:
from http import HTTPStatus
from urllib.parse import urlparse, unquote
from pathlib import PurePosixPath
import requests
from dashscope import ImageSynthesis


def simple_call():
    prompt = 'Mouse rides elephant'
    rsp = ImageSynthesis.call(model=ImageSynthesis.Models.wanx_v1,
                              prompt=prompt,
                              n=4,
                              size='1024*1024')
    if rsp.status_code == HTTPStatus.OK:
        print(rsp.output)
        print(rsp.usage)
        # save file to current directory
        for result in rsp.output.results:
            file_name = PurePosixPath(unquote(urlparse(result.url).path)).parts[-1]
            with open('./%s' % file_name, 'wb+') as f:
                f.write(requests.get(result.url).content)
    else:
        print('Failed, status_code: %s, code: %s, message: %s' %
              (rsp.status_code, rsp.code, rsp.message))


if __name__ == '__main__':
    simple_call()

{"task_id": "cae4ca0d-9a7c-4f3b-9a9c-c0a72a9a7d81", "task_status": "SUCCEEDED", "results": [{"url": "https://dashscope-result-sh.oss-cn-shanghai.aliyuncs.com/1d/a4/20240127/723609ee/8b7003e6-7fe4-4a5f-b709-5ed9259e1d2e-1.jpg?Expires=1706453852&OSSAccessKeyId=LTAI5tQZd8AEcZX6KZV4G8qL&Signature=1HL68KsPDVpJznUKUtUVw%2BOSe9I%3D"}, {"url": "https://dashscope-result-bj.oss-cn-beijing.aliyuncs.com/1d/6b/20240127/8d820c8d/8ace26c2-9f14-4e83-9184-4b2672e44a6b-1.jpg?Expires=1706453852&OSSAccessKeyId=LTAI5tQZd8AEcZX6KZV4G8qL&Signature=cuexmlETqgLsivkipAMWTGTF5kM%3D"}, {"url": "https://dashscope-result-bj.oss-cn-beijing.aliyuncs.com/1d/db/20240127/8d820c8d/848ab3df-2373-4c0f-a2bb-38d0faac381d-1.jpg?Expires=1706453852&OSSAccessKeyId=LTAI5tQZd8AEcZX6KZV4G8qL&Signature=50kzWJss86mHJopOEDpP51dPrOY%3D"}, {"url": "https://dashscope-result-bj.oss-cn-beijing.aliyuncs.com/1d/0f/20240127/8d820c8d/717c78c0-af4b-4a29-8de6-d1e885017b9a-1.jpg?Expires=1706453852&OSSAccessKeyId=LTAI5tQZd8AEcZX6KZV4G8qL&Signature

### Fake LLM

In [ ]:
!poetry add langchain_experimental

使用 **FakeListLLM** 可以模拟大模型的响应，这可以用于实现模拟演示。

In [80]:
from langchain.llms.fake import FakeListLLM
from langchain_experimental.tools import PythonREPLTool
from langchain.agents import initialize_agent
from langchain.agents import AgentType
tools = [PythonREPLTool()]
responses = [
    "Action: Python_REPL\nAction Input: print(2 + 2)",
    "Final Answer: 4"]
llm = FakeListLLM(responses=responses)
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)
agent.invoke("whats 2 + 2")



> Entering new AgentExecutor chain...
Action: Python_REPL
Action Input: print(2 + 2)
Observation: 4

Thought:Final Answer: 4

> Finished chain.


{'input': 'whats 2 + 2', 'output': '4'}

## 集成langfuse

In [2]:
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langfuse.callback import CallbackHandler
import uuid

In [4]:
handler = CallbackHandler(trace_name="learning-langchain", user_id="homeway", session_id=str(uuid.uuid4()))

In [6]:
llm = ChatOpenAI(model = "gpt-3.5-turbo", streaming = False, temperature = 0.5)
parser = StrOutputParser()
prompt = ChatPromptTemplate.from_template("hi")
train = (prompt | llm | parser)
train.invoke({}, config = {"callbacks": [handler]})

'Hello! How can I assist you today?'

In [8]:
# 多轮对话
from langchain.schema import (
    AIMessage, #等价于OpenAI接口中的 assistant role
    HumanMessage, #等价于OpenAI接口中的 user role
    SystemMessage #等价于OpenAI接口中的 system role
)

messages = [
    SystemMessage(content="你是AGIClass的课程助理。"), 
    HumanMessage(content="我是学员，我叫薛宏伟。"), 
    AIMessage(content="欢迎！"),
    HumanMessage(content="我是谁") 
]
llm.invoke(messages) 

AIMessage(content='你是薛宏伟。')

In [15]:
# 对话提示语模板
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate

template = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("你是{product}的客服助手。你的名字叫{name}"),
        HumanMessagePromptTemplate.from_template("{query}"),
    ]
)

llm = ChatOpenAI()
prompt = template.format_messages(
        product="广州鸿蒙",
        name="蒙蒙",
        query="你是谁"
    )

llm.invoke(prompt)

AIMessage(content='我是广州鸿蒙的客服助手，名字叫蒙蒙。有什么可以帮到您的吗？')

## 集成langserve

### 与fastapi一起使用

In [ ]:
#!/usr/bin/env python
from fastapi import FastAPI
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langserve import add_routes

app = FastAPI(
    title="LangChain Server",
    version="1.0",
    description="A simple api server using Langchain's Runnable interfaces",
)

add_routes(
    app,
    ChatOpenAI(),
    path="/openai",
)

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
add_routes(
    app,
    prompt | model,
    path="/joke",
)

if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="localhost", port=8000)

### 与langfuse一起使用

In [ ]:
handler = CallbackHandler(trace_name="chat_once", user_id="wencheng")
prompt = ChatPromptTemplate.from_template(
    """{question}""")
llm = ChatOpenAI(model = "gpt-3.5-turbo-16k", streaming = True, temperature = 0)
chain = (prompt | llm | parser).with_config({"callbacks": [handler]})

add_routes(app, chain, path = "/langserve/chat_once")

### python Client

In [67]:
!poetry add httpx_sse

Using version ^0.4.0 for httpx-sse

Updating dependencies
Resolving dependencies... (16.3s)Resolving dependencies... (0.1s)Resolving dependencies... (7.9s)Resolving dependencies... (9.3s)

Package operations: 1 install, 0 updates, 0 removals

  • Installing httpx-sse (0.4.0): Pending...
  • Installing httpx-sse (0.4.0): Installing...
  • Installing httpx-sse (0.4.0)

Writing lock file


In [64]:
!poetry add langserve

The following packages are already present in the pyproject.toml and will be skipped:

  • langserve

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Nothing to add.


In [70]:
from langserve import RemoteRunnable
chat_once = RemoteRunnable("http://localhost:8000/langserve/chat_once")

东莞是中国广东省下辖的一个地级市，位于珠江三角洲南部，东临深圳，西接广州，北邻惠州，南濒珠江口。作为中国改革开放的重要窗口和制造业基地，东莞是中国最重要的制造业城市之一。

东莞是中国最早的经济特区之一，也是中国最大的制造业城市之一。它以制造业为主导，涵盖了电子、电器、纺织、玩具、家具、鞋业等多个行业。许多国内外知名品牌都在东莞设有生产基地。东莞的制造业发展水平和产业链完善程度在全国具有较高的竞争力。

除了制造业，东莞也在不断发展其他产业，如现代服务业、高新技术产业和文化创意产业等。近年来，东莞还加大了对科技创新的投入，积极推动产业升级和转型发展。

东莞也是一个宜居的城市，拥有良好的基础设施和公共服务。城市规划合理，交通便利，医疗、教育、文化等公共服务设施完善。同时，东莞还注重生态环境保护，积极推动绿色发展，建设了许多公园和绿地，提供了良好的生活环境。

此外，东莞还有一些旅游景点值得一提。如虎门石龙山、广东现代国际展览中心、东莞松山湖科技产业园等。这些景点展示了东莞的自然风光和城市发展成果。

总的来说，东莞是一个以制造业为主导的现代化城市，拥有发达的经济和良好的生活环境。无论是商务出差还是旅游观光，东莞都是一个值得一去的地方。

In [71]:
chat_once.invoke({"question": "能帮我介绍一下东莞吗？"})

'东莞是中国广东省下辖的一个地级市，位于珠江三角洲南部，东临深圳，西接广州，北邻惠州，南濒珠江口。作为中国改革开放的重要窗口和制造业基地，东莞是中国最重要的制造业城市之一。\n\n东莞是中国最早的经济特区之一，也是中国最大的制造业城市之一。它以制造业为主导，涵盖了电子、电器、纺织、玩具、家具、鞋业等多个行业。许多国内外知名品牌都在东莞设有生产基地。东莞的制造业发展水平和产业链完善程度在全国具有较高的竞争力。\n\n除了制造业，东莞也在不断发展其他产业，如现代服务业、高新技术产业和文化创意产业等。近年来，东莞还加大了对科技创新的投入，积极推动产业升级和转型发展。\n\n东莞也是一个宜居的城市，拥有良好的基础设施和公共服务。城市规划合理，交通便利，医疗、教育、文化等公共服务设施完善。同时，东莞还注重生态环境保护，积极推动绿色发展，建设了许多公园和绿地，提供了良好的生活环境。\n\n此外，东莞还有一些旅游景点值得一提。如虎门石龙山、广东现代国际展览中心、东莞松山湖科技产业园等。这些景点展示了东莞的自然风光和城市发展成果。\n\n总的来说，东莞是一个以制造业为主导的现代化城市，拥有发达的经济和良好的生活环境。无论是商务出差还是旅游观光，东莞都是一个值得一去的地方。'

In [ ]:
for chunk in chat_once.stream({"question": "能帮我介绍一下东莞吗？"}):
    print(chunk, end="", flush=True)

### javascript client

In [ ]:
!yarn add langchain

#### 调用invoke

In [ ]:
import { RemoteRunnable } from "langchain/runnables/remote";

const remoteChain = new RemoteRunnable({
  url: "https://your_hostname.com/path",
});

const result = await remoteChain.invoke({
  param1: "param1",
  param2: "param2",
});

console.log(result);

#### 调用stream

In [ ]:
const stream = await remoteChain.stream({
  param1: "param1",
  param2: "param2",
});

for await (const chunk of stream) {
  console.log(chunk);
}

#### 使用config

In [ ]:
import { RemoteRunnable } from "langchain/runnables/remote";

const remoteChain = new RemoteRunnable({
  url: "https://your_hostname.com/path",
  options: {
    timeout: 10000,
    headers: {
      Authorization: "Bearer YOUR_TOKEN",
    },
  },
});

const result = await remoteChain.invoke({
  param1: "param1",
  param2: "param2",
});

console.log(result);